# Análise de Séries Temporais - Whitebook: Projeto Clima e Saúde

In [0]:
#-- %pip install openpyxl
#-- %pip install geobr
#-- %pip install geobr matplotlib matplotlib-scalebar


In [0]:
%restart_python

## Importação

### Importando pacotes

In [0]:
# Importando pacotes usados na análise
from datetime import datetime
from joblib import Parallel, delayed
from keras.layers import LSTM, Dense
from keras.models import Sequential
from matplotlib_scalebar.scalebar import ScaleBar
from pmdarima import auto_arima
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F, SparkSession
from pyspark.sql.functions import col, concat, countDistinct, create_map, current_date, datediff, expr, length, lit, lower, max, min, regexp_extract, regexp_replace, sum, to_date, trim, udf, when    
from pyspark.sql.types import DateType, IntegerType, StringType
from pyspark.sql.window import Window
from scipy import stats
from scipy.stats import f_oneway, norm,pearsonr, shapiro, zscore  
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, explained_variance_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

import glob
import itertools
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np 
import pandas as pd
import pmdarima as pm
import os
import random
import seaborn as sns
import statsmodels.api as sm
import matplotlib.dates as mdates
import xgboost as xgb

2024-12-03 20:08:14.258531: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 20:08:14.364931: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 20:08:14.818723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 20:08:19.571783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024/12/03 20:08:21 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for tensorflow: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/databricks/python/lib/python3.11/si

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2897926076219238>, line 6
      4 from keras.layers import LSTM, Dense
      5 from keras.models import Sequential
----> 6 from matplotlib_scalebar.scalebar import ScaleBar
      7 from pmdarima import auto_arima
      8 from prophet import Prophet

ModuleNotFoundError: No module named 'matplotlib_scalebar'

### Importando dados

In [0]:
# Importando dados 
cid = spark.read.table("data_research_dev.raw_cid10.rotulos_cid_10")
cid_pandas = cid.toPandas()

sih_spark = spark.read.table("data_research_dev.raw_sih_sensiveis_clima.sih_microdados_individuais_2018_2023")
sih_pandas = sih_spark.toPandas()

sih_spark_group = spark.read.table("data_research_dev.raw_sih_sensiveis_clima.sih_dados_agrupados_municipios_2018_2023")
sih_pandas_group = sih_spark_group.toPandas()

wb_micro_spark = spark.read.table("data_research_dev.raw_acessos_wb.afya_grupotemas_wb_brasil")
wb_micro_pandas = wb_micro_spark.toPandas()

wb_micro_spark_group = spark.read.table("data_research_dev.raw_acessos_wb.afya_grupotemas_wb_microrregioes_arupado")
wb_micro_pandas_group = wb_micro_spark_group.toPandas()

wb_micro_spark_uf_group = spark.read.table("data_research_dev.raw_acessos_wb.afya_grupotemas_wb_estado")
wb_micro_pandas_uf_group = wb_micro_spark_uf_group.toPandas()

correspondencia_spark_cid = spark.read.table("data_research_dev.raw_cids_sensiveis_clima.lista_cids_selecionados_correspondencia_wb")
correspondencia_pandas_cid = correspondencia_spark_cid.toPandas()

projecao_populacional_spark = spark.read.table("data_research_dev.populacao_ibge.raw_projecao_populacao_ibge_uf")
projecao_populacional_pandas = projecao_populacional_spark.toPandas()

projecao_nacional_spark = spark.read.table("data_research_dev.populacao_ibge.raw_projecao_populacao_ibge")
projecao_nacional_pandas = projecao_nacional_spark.toPandas()

com.databricks.backend.common.rpc.DriverStoppedException: Driver down cause: driver state change (exit code: 15)
	at com.databricks.spark.chauffeur.ChauffeurState.processDriverStateChange(ChauffeurState.scala:492)
	at com.databricks.spark.chauffeur.Chauffeur.onDriverStateChange(Chauffeur.scala:1490)
	at com.databricks.spark.chauffeur.Chauffeur.$anonfun$driverStateOpt$1(Chauffeur.scala:187)
	at com.databricks.spark.chauffeur.Chauffeur.$anonfun$driverStateOpt$1$adapted(Chauffeur.scala:187)
	at com.databricks.spark.chauffeur.DriverDaemonMonitorImpl.$anonfun$goToStopped$4(DriverDaemonMonitorImpl.scala:273)
	at com.databricks.spark.chauffeur.DriverDaemonMonitorImpl.$anonfun$goToStopped$4$adapted(DriverDaemonMonitorImpl.scala:273)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.spark.chauffeur.DriverDaemonMonitorImpl.goToStopped(DriverDaemonMonitorImpl.scala:273)
	at com.databricks.spark.chauffeur.DriverDaemonMonitorImpl.monitorDriver(DriverDaemonMonitorImpl.sc

## Análise Descritiva

### Analise por sexo e faixa etária

In [0]:
# Função para criar a coluna de faixa etária
def criar_faixa_etaria(df, coluna_idade, nova_coluna):
    # Definindo os intervalos e os rótulos das faixas etárias
    bins = [-1, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, float("inf")]
    labels = ["<1", "1-5", "6-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-90", "90+"]
    # Criando a nova coluna
    df[nova_coluna] = pd.cut(df[coluna_idade], bins=bins, labels=labels, right=True)
    return df

In [0]:
# Criar a faixa etária na coluna 'FAIXA_ETARIA'
sih_pandas = criar_faixa_etaria(sih_pandas, coluna_idade="IDADE", nova_coluna="FAIXA_ETARIA")


In [0]:
# Criar a Tabela 1: Linhas (FAIXA_ETARIA) x Colunas (SEXO)
tabela_1 = (
    sih_pandas.groupby(["FAIXA_ETARIA", "SEXO"])["N_AIH"]
    .nunique()  # Contar valores únicos de N_AIH
    .reset_index()  # Resetar o índice para facilitar o pivot
    .pivot(index="FAIXA_ETARIA", columns="SEXO", values="N_AIH")  # Criar a tabela no formato solicitado
)

# Criar a Tabela 2: Linhas (DIAG_PRINC_NOME) x Colunas (SEXO)
tabela_2 = (
    sih_pandas.groupby(["DIAG_PRINC_NOME", "SEXO"])["N_AIH"]
    .nunique()  # Contar valores únicos de N_AIH
    .reset_index()  # Resetar o índice para facilitar o pivot
    .pivot(index="DIAG_PRINC_NOME", columns="SEXO", values="N_AIH")  # Criar a tabela no formato solicitado
)

# Criar a Tabela 3: Linhas (DIAG_PRINC_NOME) x Colunas (FAIXA_ETARIA)
tabela_3 = (
    sih_pandas.groupby(["DIAG_PRINC_NOME", "FAIXA_ETARIA"])["N_AIH"]
    .nunique()  # Contar valores únicos de N_AIH
    .reset_index()  # Resetar o índice para facilitar o pivot
    .pivot(index="DIAG_PRINC_NOME", columns="FAIXA_ETARIA", values="N_AIH")  # Criar a tabela no formato solicitado
)


In [0]:
# Função para adicionar total e calcular frequência e percentual
def adicionar_totais_e_percentuais(tabela, total_label="Total"):
    # Adicionar a linha com os totais
    tabela.loc[total_label] = tabela.sum(axis=0)
    
    # Calcular os percentuais
    percentuais = tabela.div(tabela.loc[total_label], axis=1) * 100
    
    # Renomear colunas para incluir frequência e percentual
    tabela_final = pd.concat([tabela, percentuais], axis=1)
    tabela_final.columns = [f"{col}_frequencia" if i < len(tabela.columns) / 2 else f"{col}_percentual" 
                            for i, col in enumerate(tabela_final.columns)]
    
    return tabela_final

In [0]:
# Recriar as tabelas com totais e percentuais

# Faixa Etária x Sexo
tabela_1_com_totais = adicionar_totais_e_percentuais(tabela_1)

# Diagnóstico x Sexo
tabela_2_com_totais = adicionar_totais_e_percentuais(tabela_2)

# Diagnóstico x Faixa Etária
tabela_3_com_totais = adicionar_totais_e_percentuais(tabela_3)


In [0]:
# Salvando as tabelas no caminho especificado
caminho_base = "/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_descritiva_sih/dados_demograficos/"

# Tabela 1: Faixa Etária x Sexo
tabela_1_caminho = f"{caminho_base}tabela_faixaetaria_sexo_sih.xlsx"
tabela_1_com_totais.to_excel(tabela_1_caminho, index=True)

# Tabela 2: Diagnóstico x Sexo
tabela_2_caminho = f"{caminho_base}tabela_diagnostico_sexo_sih.xlsx"
tabela_2_com_totais.to_excel(tabela_2_caminho, index=True)

# Tabela 3: Diagnóstico x Faixa Etária
tabela_3_caminho = f"{caminho_base}tabela_diagnostico_faixaetaria_sih.xlsx"
tabela_3_com_totais.to_excel(tabela_3_caminho, index=True)




In [0]:
# Agrupar os dados por Faixa Etária e Sexo e contar os valores únicos de N_AIH
piramide_etaria = sih_pandas.groupby(["FAIXA_ETARIA", "SEXO"])["N_AIH"].nunique().unstack(fill_value=0)

# Separar os dados por sexo
masculino = piramide_etaria.get("M", pd.Series(index=piramide_etaria.index, data=piramide_etaria['Masculino']))
feminino = piramide_etaria.get("F", pd.Series(index=piramide_etaria.index, data = piramide_etaria['Feminino']))


# Configuração para o gráfico
faixas_etarias = piramide_etaria.index.astype(str)  # Converter índices para string
posicoes = np.arange(len(faixas_etarias))

# Criar o gráfico de pirâmide etária
plt.figure(figsize=(10, 8))
plt.barh(posicoes, -masculino, color="blue", label="Masculino")  # Valores negativos para a esquerda
plt.barh(posicoes, feminino, color="pink", label="Feminino")

# Configurações adicionais
plt.yticks(posicoes, faixas_etarias)
plt.xlabel("Número de Internações")
plt.ylabel("Faixa Etária")
plt.title("Pirâmide Etária de Internações")
plt.legend(loc="best")
plt.axvline(0, color="black", linewidth=0.5)  # Linha central

# Configurar o formato do eixo X para exibir números normalmente
plt.gca().xaxis.set_major_formatter(mticker.StrMethodFormatter("{x:.0f}"))

plt.tight_layout()

# Salvando a figura
plt.savefig("/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_descritiva_sih/graficos/faixa_etaria.png")
plt.close()

# Exibir o gráfico
plt.show()


In [0]:
%python
%pip install xlsxwriter

import pandas as pd
from pandas import ExcelWriter
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os

# Inicializando o DataFrame sih_spark
sih_spark = spark.read.table("data_research_dev.raw_sih_sensiveis_clima.sih_microdados_individuais_2018_2023")
populacao_df = spark.read.table("data_research_dev.populacao_ibge.raw_projecao_populacao_ibge")
populacao_df = populacao_df.withColumnRenamed("ANO", "ANO_POP").withColumnRenamed("POPULACAO", "POPULACAO_ANO")

# Garantir que a coluna ANO está presente em sih_spark
sih_spark = sih_spark.withColumn("ANO", F.year(F.col("DT_INTER")))

# Etapa 1: Agregar corretamente N_AIH
sih_spark = sih_spark.groupBy("DIAG_PRINC_NOME", "ANO").agg(F.count("N_AIH").alias("N_AIH"))

# Etapa 2: Calcular o total de internações por ano
window_ano = Window.partitionBy("ANO")
sih_spark = sih_spark.withColumn("Total_Ano", F.sum("N_AIH").over(window_ano))

# Etapa 3: Calcular percentual de internações por diagnóstico e ano
sih_spark = sih_spark.withColumn("Percentual", F.round((F.col("N_AIH") / F.col("Total_Ano")) * 100, 2))

# Etapa 4: Adicionar dados populacionais e calcular taxa
sih_spark = sih_spark.join(populacao_df, sih_spark["ANO"] == populacao_df["ANO_POP"], "left")
sih_spark = sih_spark.withColumn("Taxa", F.round((F.col("N_AIH") / F.col("POPULACAO_ANO")) * 100000, 2))

# Etapa 5: Incorporar Diagnóstico Secundário
diagnosticos_sec_df = spark.read.table("data_research_dev.raw_cids_sensiveis_clima.lista_cids_selecionados_correspondencia_wb")
diagnosticos_sec_df = diagnosticos_sec_df.select("DIAG_PRINC_NOME", "CATEGORIA_SECUNDARIA")
sih_spark = sih_spark.join(diagnosticos_sec_df, on="DIAG_PRINC_NOME", how="left")

# Etapa 6: Reorganizar tabela para o formato desejado
tabela_diagnostico_secundario = (
    sih_spark
    .select("DIAG_PRINC_NOME", "CATEGORIA_SECUNDARIA", "ANO", "N_AIH", "Percentual", "Taxa")
    .withColumnRenamed("N_AIH", "n")
    .groupBy("DIAG_PRINC_NOME", "CATEGORIA_SECUNDARIA")
    .pivot("ANO", [2021, 2022, 2023, 2024])
    .agg(F.first("n").alias("n"), F.first("Percentual").alias("Percentual"), F.first("Taxa").alias("Taxa"))
)

# Converta o DataFrame para Pandas
tabela_pandas = tabela_diagnostico_secundario.toPandas()

# Salvando as tabelas no caminho especificado
caminho_base = "/Workspace/Users/dayanna.quintanilha@afya.com.br/"

# Ensure the directory exists
os.makedirs(caminho_base, exist_ok=True)

# Tabela 4: Diagnóstico x Ano com Diagnóstico Secundário
tabela_4_caminho = f"{caminho_base}tabela_diagnostico_secundario_sih.xlsx"

# Salvando a Tabela 4 no caminho especificado
with ExcelWriter(tabela_4_caminho, engine="xlsxwriter") as writer:
    tabela_pandas.to_excel(writer, index=False, sheet_name="Tabela Diagnóstico Secundário")
    writer.save()

print(f"Tabela 4 salva em: {tabela_4_caminho}")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


/root/.ipykernel/1125/command-1204941237348366-3568575663:61: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/local_disk0/.ephemeral_nfs/envs/pythonEnv-5823f045-7c69-4d5b-a3a3-18eeb792523e/lib/python3.11/site-packages/xlsxwriter/workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Tabela 4 salva em: /Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_diagnostico_secundario_sih.xlsx


In [0]:
from pyspark.sql import functions as F

# Agrupando por Categoria Secundária e ajustando a tabela
tabela_reformatada = (
    tabela_diagnostico_secundario
    .groupBy("CATEGORIA_SECUNDARIA")
    .agg(
        F.sum("2021_n").alias("2021_n"),
        F.avg("2021_Percentual").alias("2021_%"),
        F.avg("2021_Taxa").alias("2021_Taxa (100.000 hab)"),
        F.sum("2022_n").alias("2022_n"),
        F.avg("2022_Percentual").alias("2022_%"),
        F.avg("2022_Taxa").alias("2022_Taxa (100.000 hab)"),
        F.sum("2023_n").alias("2023_n"),
        F.avg("2023_Percentual").alias("2023_%"),
        F.avg("2023_Taxa").alias("2023_Taxa (100.000 hab)"),
        F.sum("2024_n").alias("2024_n"),
        F.avg("2024_Percentual").alias("2024_%"),
        F.avg("2024_Taxa").alias("2024_Taxa (100.000 hab)")
    )
    .orderBy("CATEGORIA_SECUNDARIA")
)

# Convertendo para Pandas para ajustes no formato
tabela_pandas = tabela_reformatada.toPandas()

# Ajustando o cabeçalho para strings simples
tabela_pandas.columns = [
    "Categoria Secundária",
    "2021_n", "2021_%", "2021_Taxa (100.000 hab)",
    "2022_n", "2022_%", "2022_Taxa (100.000 hab)",
    "2023_n", "2023_%", "2023_Taxa (100.000 hab)",
    "2024_n", "2024_%", "2024_Taxa (100.000 hab)"
]

# Salvando os arquivos
caminho_csv = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.csv"
caminho_excel = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.xlsx"

# Salvando em CSV
tabela_pandas.to_csv(caminho_csv, index=False, encoding="utf-8")


In [0]:
from pyspark.sql import functions as F

# Agrupando por Categoria Secundária
tabela_reformatada = (
    tabela_diagnostico_secundario
    .groupBy("CATEGORIA_SECUNDARIA")
    .agg(
        F.sum("2021_n").alias("2021_n"),
        F.avg("2021_Percentual").alias("2021_%"),
        F.avg("2021_Taxa").alias("2021_Taxa (100.000 hab)"),
        F.sum("2022_n").alias("2022_n"),
        F.avg("2022_Percentual").alias("2022_%"),
        F.avg("2022_Taxa").alias("2022_Taxa (100.000 hab)"),
        F.sum("2023_n").alias("2023_n"),
        F.avg("2023_Percentual").alias("2023_%"),
        F.avg("2023_Taxa").alias("2023_Taxa (100.000 hab)"),
        F.sum("2024_n").alias("2024_n"),
        F.avg("2024_Percentual").alias("2024_%"),
        F.avg("2024_Taxa").alias("2024_Taxa (100.000 hab)")
    )
    .orderBy("CATEGORIA_SECUNDARIA")
)

# Verificar o conteúdo após o agrupamento
tabela_reformatada.show(10, truncate=False)
print(f"Total de registros após agrupamento: {tabela_reformatada.count()}")

# Convertendo para Pandas para ajustes no formato
tabela_pandas = tabela_reformatada.toPandas()

# Verificar o conteúdo do DataFrame Pandas
print(tabela_pandas.head())
print(f"Total de registros no DataFrame Pandas: {len(tabela_pandas)}")

# Ajustando o cabeçalho para strings simples
tabela_pandas.columns = [
    "Categoria Secundária",
    "2021_n", "2021_%", "2021_Taxa (100.000 hab)",
    "2022_n", "2022_%", "2022_Taxa (100.000 hab)",
    "2023_n", "2023_%", "2023_Taxa (100.000 hab)",
    "2024_n", "2024_%", "2024_Taxa (100.000 hab)"
]

# Salvando os arquivos
caminho_csv = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.csv"
caminho_excel = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.xlsx"

# Salvando em CSV
tabela_pandas.to_csv(caminho_csv, index=False, encoding="utf-8")

# Salvando em Excel
with pd.ExcelWriter(caminho_excel, engine="xlsxwriter") as writer:
    tabela_pandas.to_excel(writer, index=False, sheet_name="Tabela Agravo x Ano")

print(f"Tabela salva no formato solicitado em CSV: {caminho_csv}")
print(f"Tabela salva no formato solicitado em Excel: {caminho_excel}")


+------------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+
|CATEGORIA_SECUNDARIA    |2021_n|2021_%             |2021_Taxa (100.000 hab)|2022_n|2022_%             |2022_Taxa (100.000 hab)|2023_n|2023_%             |2023_Taxa (100.000 hab)|2024_n|2024_%             |2024_Taxa (100.000 hab)|
+------------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+------+-------------------+-----------------------+
|NULL                    |82166 |0.09512820512820509|0.9871794871794873     |134305|0.19634146341463413|1.52390243902439       |148219|0.2314999999999999 |1.7135000000000002     |201357|0.4917948717948721 |2.371025641025642      |
|Arboviroses             |295   |0.01               |0.14                   

In [0]:
import pandas as pd

# Certificar-se de que access_date é do tipo datetime
wb_micro_pandas_uf_group['access_date'] = pd.to_datetime(wb_micro_pandas_uf_group['access_date'], errors='coerce')

# Criar uma nova coluna para o ano
wb_micro_pandas_uf_group['ano'] = wb_micro_pandas_uf_group['access_date'].dt.year

# Verificar os dados carregados
print("Primeiras linhas do DataFrame após adicionar o ano:\n", wb_micro_pandas_uf_group.head())

# Filtrar os anos relevantes
anos_relevantes = [2021, 2022, 2023, 2024]
wb_micro_pandas_uf_group = wb_micro_pandas_uf_group[wb_micro_pandas_uf_group['ano'].isin(anos_relevantes)]

# Pivotar os dados para criar a tabela com os acessos por estado e ano
tabela_acessos = wb_micro_pandas_uf_group.pivot_table(
    index='access_state',  # Coluna representando os estados
    columns='ano',         # Coluna representando os anos
    values='count',        # Coluna representando o número de acessos
    aggfunc='sum',         # Somar os acessos por estado e ano
    fill_value=0           # Substituir valores ausentes por 0
)

# Renomear as colunas para garantir que os anos sejam strings
tabela_acessos.columns = tabela_acessos.columns.astype(str)

# Resetar o índice para que access_state se torne uma coluna
tabela_acessos.reset_index(inplace=True)

# Adicionar uma coluna de soma total de acessos
tabela_acessos['Total'] = tabela_acessos.iloc[:, 1:].sum(axis=1)

# Ordenar pela soma total de acessos em ordem decrescente
tabela_acessos = tabela_acessos.sort_values(by='Total', ascending=False)

# Salvar os resultados em Excel
caminho_excel = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.xlsx"
tabela_acessos.to_excel(caminho_excel, index=False, sheet_name="Acessos por Ano")

print(f"Tabela de análise de acessos por ano salva em: {output_path}")

# Exibir a tabela final
print(tabela_acessos)


Primeiras linhas do DataFrame após adicionar o ano:
   access_date                      content_main_theme access_state  count   ano
0  2021-01-01                                    Asma           SP     29  2021
1  2021-01-01      Doença pulmonar obstrutiva crônica           PE      2  2021
2  2021-01-01                                Covid-19           PE     14  2021
3  2021-01-01                                Covid-19           RS     32  2021
4  2021-01-01  Acidente vascular encefálico isquêmico           SP     26  2021


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-3650798557410422>, line 39
     37 # Salvar os resultados em Excel
     38 caminho_excel = "/Workspace/Users/dayanna.quintanilha@afya.com.br/tabela_agravo_formatada.xlsx"
---> 39 tabela_acessos.to_excel(caminho_excel, index=False, sheet_name="Acessos por Ano")
     41 print(f"Tabela de análise de acessos por ano salva em: {output_path}")
     43 # Exibir a tabela final

File /databricks/python/lib/python3.11/site-packages/pandas/util/_decorators.py:211, in deprecate_kwarg.<locals>._deprecate_kwarg.<locals>.wrapper(*args, **kwargs)
    209     else:
    210         kwargs[new_arg_name] = new_arg_value
--> 211 return func(*args, **kwargs)

File /databricks/python/lib/python3.11/site-packages/pandas/util/_decorators.py:211, in deprecate_kwarg.<locals>._deprecate_kwarg.<locals>.wrapper(*args, **kwargs)
    209     else:
    21

In [0]:
%python
# Install the openpyxl module
%pip install openpyxl

import pandas as pd
import os
from pandas import ExcelWriter

# Certificar-se de que access_date é do tipo datetime
wb_micro_pandas_uf_group['access_date'] = pd.to_datetime(wb_micro_pandas_uf_group['access_date'], errors='coerce')

# Criar uma nova coluna para o ano
wb_micro_pandas_uf_group['ano'] = wb_micro_pandas_uf_group['access_date'].dt.year

# Verificar os dados carregados
print("Primeiras linhas do DataFrame após adicionar o ano:\n", wb_micro_pandas_uf_group.head())

# Filtrar os anos relevantes
anos_relevantes = [2021, 2022, 2023, 2024]
wb_micro_pandas_uf_group = wb_micro_pandas_uf_group[wb_micro_pandas_uf_group['ano'].isin(anos_relevantes)]

# Pivotar os dados para criar a tabela com os acessos por estado e ano
tabela_acessos = wb_micro_pandas_uf_group.pivot_table(
    index='access_state',  # Coluna representando os estados
    columns='ano',         # Coluna representando os anos
    values='count',        # Coluna representando o número de acessos
    aggfunc='sum',         # Somar os acessos por estado e ano
    fill_value=0           # Substituir valores ausentes por 0
)

# Renomear as colunas para garantir que os anos sejam strings
tabela_acessos.columns = tabela_acessos.columns.astype(str)

# Resetar o índice para que access_state se torne uma coluna
tabela_acessos.reset_index(inplace=True)

# Adicionar uma coluna de soma total de acessos
tabela_acessos['Total'] = tabela_acessos.iloc[:, 1:].sum(axis=1)

# Ordenar pela soma total de acessos em ordem decrescente
tabela_acessos = tabela_acessos.sort_values(by='Total', ascending=False)

# Converta o DataFrame para Pandas
tabela_pandas = tabela_acessos

# Salvando as tabelas no caminho especificado
caminho_base = "/Workspace/Users/dayanna.quintanilha@afya.com.br/"

# Ensure the directory exists
os.makedirs(caminho_base, exist_ok=True)

# Tabela 5: Diagnóstico x Ano com Diagnóstico Secundário
tabela_5_caminho = f"{caminho_base}acessos_WB_estado.xlsx"

# Salvando a Tabela 5 no caminho especificado
with ExcelWriter(tabela_5_caminho, engine="xlsxwriter") as writer:
    tabela_pandas.to_excel(writer, index=False, sheet_name="Acessos WB")
    writer.save()

print(f"Tabela 5 salva em: {tabela_5_caminho}")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Primeiras linhas do DataFrame após adicionar o ano:
   access_date                      content_main_theme access_state  count   ano
0  2021-01-01                                    Asma           SP     29  2021
1  2021-01-01      Doença pulmonar obstrutiva crônica           PE      2  2021
2  2021-01-01                                Covid-19           PE     14  2021
3  2021-01-01                                Covid-19           RS     32  2021
4  2021-01-01  Acidente vascular encefálico isquêmico           SP     26  2021


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-3650798557410437>, line 55
     52 tabela_5_caminho = f"{caminho_base}acessos_WB_estado.xlsx"
     54 # Salvando a Tabela 5 no caminho especificado
---> 55 with ExcelWriter(tabela_5_caminho, engine="xlsxwriter") as writer:
     56     tabela_pandas.to_excel(writer, index=False, sheet_name="Acessos WB")
     57     writer.save()

File /databricks/python/lib/python3.11/site-packages/pandas/io/excel/_xlsxwriter.py:198, in XlsxWriter.__init__(self, path, engine, date_format, datetime_format, mode, storage_options, if_sheet_exists, engine_kwargs, **kwargs)
    185 def __init__(
    186     self,
    187     path: FilePath | WriteExcelBuffer | ExcelWriter,
   (...)
    196 ) -> None:
    197     # Use the xlsxwriter module as the Excel writer.
--> 198     from xlsxwriter import Workbook
    200     engine_kwargs = combine_kwargs

In [0]:
%python
# Install the required modules
%pip install openpyxl
%pip install xlsxwriter

import pandas as pd
import os
from pandas import ExcelWriter

# Certificar-se de que access_date é do tipo datetime
wb_micro_pandas_uf_group['access_date'] = pd.to_datetime(wb_micro_pandas_uf_group['access_date'], errors='coerce')

# Criar uma nova coluna para o ano
wb_micro_pandas_uf_group['ano'] = wb_micro_pandas_uf_group['access_date'].dt.year

# Verificar os dados carregados
print("Primeiras linhas do DataFrame após adicionar o ano:\n", wb_micro_pandas_uf_group.head())

# Filtrar os anos relevantes
anos_relevantes = [2021, 2022, 2023, 2024]
wb_micro_pandas_uf_group = wb_micro_pandas_uf_group[wb_micro_pandas_uf_group['ano'].isin(anos_relevantes)]

# Pivotar os dados para criar a tabela com os acessos por estado e ano
tabela_acessos = wb_micro_pandas_uf_group.pivot_table(
    index='access_state',  # Coluna representando os estados
    columns='ano',         # Coluna representando os anos
    values='count',        # Coluna representando o número de acessos
    aggfunc='sum',         # Somar os acessos por estado e ano
    fill_value=0           # Substituir valores ausentes por 0
)

# Renomear as colunas para garantir que os anos sejam strings
tabela_acessos.columns = tabela_acessos.columns.astype(str)

# Resetar o índice para que access_state se torne uma coluna
tabela_acessos.reset_index(inplace=True)

# Adicionar uma coluna de soma total de acessos
tabela_acessos['Total'] = tabela_acessos.iloc[:, 1:].sum(axis=1)

# Ordenar pela soma total de acessos em ordem decrescente
tabela_acessos = tabela_acessos.sort_values(by='Total', ascending=False)

# Converta o DataFrame para Pandas
tabela_pandas = tabela_acessos

# Salvando as tabelas no caminho especificado
caminho_base = "/Workspace/Users/dayanna.quintanilha@afya.com.br/"

# Ensure the directory exists
os.makedirs(caminho_base, exist_ok=True)

# Tabela 5: Diagnóstico x Ano com Diagnóstico Secundário
tabela_5_caminho = f"{caminho_base}acessos_WB_estado.xlsx"

# Salvando a Tabela 5 no caminho especificado
with ExcelWriter(tabela_5_caminho, engine="xlsxwriter") as writer:
    tabela_pandas.to_excel(writer, index=False, sheet_name="Acessos WB")
    writer.save()

print(f"Tabela 5 salva em: {tabela_5_caminho}")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/159.9 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Primeiras linhas do DataFrame após adicionar o ano:
   access_date                      content_main_theme access_state  count   ano
0  2021-01-01      

/root/.ipykernel/1045/command-3650798557410443-877087719:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Tabela 5 salva em: /Workspace/Users/dayanna.quintanilha@afya.com.br/acessos_WB_estado.xlsx


### Ajuste de dados para a TS


In [0]:
#Transformando datas
sih_pandas_group['DT_INTER'] = pd.to_datetime(sih_pandas_group['DT_INTER'], format="%Y-%m-%d")

sih_pandas['DT_INTER'] = pd.to_datetime(sih_pandas['DT_INTER'], format="%Y-%m-%d")
sih_pandas['DT_SAIDA'] = pd.to_datetime(sih_pandas['DT_SAIDA'], format="%Y-%m-%d")
sih_pandas['NASC'] = pd.to_datetime(sih_pandas['NASC'], format="%Y-%m-%d")

wb_micro_pandas['access_date'] = pd.to_datetime(wb_micro_pandas['access_date'], format="%Y-%m-%d")
wb_micro_pandas_group['data_acesso'] = pd.to_datetime(wb_micro_pandas_group['data_acesso'], format="%Y-%m-%d")

In [0]:
# Tratando dados de SIH com o dados de WB para união
sih_spark_group = sih_spark_group.withColumn('DT_INTER', F.to_date(F.col('DT_INTER'), 'yyyy-MM-dd'))
wb_micro_spark_group = wb_micro_spark_group.withColumn('data_acesso', F.to_date(F.col('data_acesso'), 'yyyy-MM-dd'))

# Selecionar apenas as colunas necessárias de 'correspondencia_spark_cid'
correspondencia_spark_cid_selected = correspondencia_spark_cid.select('DIAG_PRINC', 'CATEGORIA_PRIMARIA')

# Tratando dados para extração
sih_spark_group = sih_spark_group.withColumn("DIAG_PRINC", 
                                             when(length(col("DIAG_PRINC")) == 3, 
                                                  regexp_replace(col("DIAG_PRINC"), "$", "0"))  # Adicionar 0 ao final se tiver 3 dígitos
                                             .otherwise(col("DIAG_PRINC")))
correspondencia_spark_cid_selected = correspondencia_spark_cid_selected.withColumn("DIAG_PRINC", 
                                             when(length(col("DIAG_PRINC")) == 3, 
                                                  regexp_replace(col("DIAG_PRINC"), "$", "0"))  # Adicionar 0 ao final se tiver 3 dígitos
                                             .otherwise(col("DIAG_PRINC")))

# Fazer o merge entre 'sih_spark_group' e 'correspondencia_spark_cid_selected' (similar ao merge no pandas)
sih_spark_group_cid = sih_spark_group.join(correspondencia_spark_cid_selected, on='DIAG_PRINC', how='inner')

In [0]:
# Ajustar o valor na coluna "CATEGORIA_PRIMARIA"
sih_spark_group_cid = sih_spark_group_cid.withColumn(
    "CATEGORIA_PRIMARIA",
    when(col("CATEGORIA_PRIMARIA") == "Febre de Chikungunya", "Febre Chikungunya")
    .otherwise(col("CATEGORIA_PRIMARIA"))
)

In [0]:
# Obter os valores únicos das duas colunas sem RDDs
wb_tema_principal_unicos = [row[0] for row in wb_micro_spark_uf_group.select("content_main_theme").distinct().collect()]
sih_categoria_primaria_unicos = [row[0] for row in sih_spark_group_cid.select("CATEGORIA_PRIMARIA").distinct().collect()]

# Identificar os valores que dão match entre as duas listas
valores_match = list(set(wb_tema_principal_unicos) & set(sih_categoria_primaria_unicos))

# Identificar os valores que não dão match (estão em uma lista mas não na outra)
valores_nao_match_wb = list(set(wb_tema_principal_unicos) - set(sih_categoria_primaria_unicos))
valores_nao_match_sih = list(set(sih_categoria_primaria_unicos) - set(wb_tema_principal_unicos))

# Exibir os resultados com a quantidade e os valores
print("Valores que dão match ({}): {}".format(len(valores_match), valores_match))
print("Valores únicos estaduais do Whitebook que não têm match ({}): {}".format(len(valores_nao_match_wb), valores_nao_match_wb))
print("Valores únicos do SIH que não têm match ({}): {}".format(len(valores_nao_match_sih), valores_nao_match_sih))

Valores que dão match (19): ['Infecção de trato urinário', 'Diabetes mellitus tipo 2', 'Covid-19', 'Hipertermia', 'Malária', 'Depressão', 'Dengue', 'Injúria Renal Aguda', 'Doença pulmonar obstrutiva crônica', 'Pneumonia comunitária', 'Febre Zika', 'Exaustão pelo Calor', 'Oropouche', 'Transtorno de ansiedade generalizada', 'Infarto agudo do miocárdio', 'Cetoacidose diabética', 'Asma', 'Febre Chikungunya', 'Doença renal crônica']
Valores únicos estaduais do Whitebook que não têm match (4): ['Arboviroses (Pediatria)', 'Asma (Pediatria)', 'Acidente vascular encefálico hemorrágico', 'Acidente vascular encefálico isquêmico']
Valores únicos do SIH que não têm match (6): ['Influenza', 'Pneumonia viral', 'Pneumonia bacteriana', 'Pneumonia por outros organismos', 'Insuficiência cardíaca ', 'Tuberculose']


In [0]:
# Criar cópia tratada de 'CATEGORIA_PRIMARIA' em 'sih_spark_group_cid'
sih_spark_group_cid = sih_spark_group_cid.withColumn(
    "CATEGORIA_PRIMARIA_TRATADA",
    trim(
        regexp_replace(
            lower(
                regexp_replace(col("CATEGORIA_PRIMARIA"), "[^a-zA-Z0-9 ]", "")
            ), 
            " {2,}", " "  # Substituir múltiplos espaços por um único espaço
        )
    )
)

# Criar cópia tratada de 'tema_principal' em 'wb_micro_spark_group'
wb_micro_spark_group = wb_micro_spark_group.withColumn(
    "tema_principal_tratado",
    trim(
        regexp_replace(
            lower(
                regexp_replace(col("tema_principal"), "[^a-zA-Z0-9 ]", "")
            ), 
            " {2,}", " "  # Substituir múltiplos espaços por um único espaço
        )
    )
)

# Fazer o merge (join) usando as colunas tratadas
sih_spark_group_wb = sih_spark_group_cid.join(
    wb_micro_spark_group,
    (sih_spark_group_cid['DT_INTER'] == wb_micro_spark_group['data_acesso']) & 
    (sih_spark_group_cid['CATEGORIA_PRIMARIA_TRATADA'] == wb_micro_spark_group['tema_principal_tratado']),
    how='inner'
)

# Remover as colunas tratadas após o merge
sih_spark_group_wb = sih_spark_group_wb.drop("CATEGORIA_PRIMARIA_TRATADA", "tema_principal_tratado")

In [0]:
# Agrupar os dados por 'DT_INTER' e 'tema_principal', somando 'quantidade_acessos' e 'N_AIH'
dados_agrupados = sih_spark_group_wb.groupBy('DT_INTER', 'tema_principal').agg(
    F.sum('quantidade_acessos').alias('total_acessos'),
    F.sum('N_AIH').alias('total_internacoes')
)

# Ordenar os dados por 'DT_INTER' para garantir a sequência temporal
dados_agrupados = dados_agrupados.orderBy('DT_INTER')

In [0]:
# Converter para Pandas DataFrame para criar gráficos
dados_agrupados_pandas= dados_agrupados.toPandas()

In [0]:
# Criar a coluna 'ano' em dados_agrupados_pandas a partir da coluna 'DT_INTER'
dados_agrupados_pandas['ano'] = pd.to_datetime(dados_agrupados_pandas['DT_INTER']).dt.year

In [0]:
# Garantir que a coluna 'ano' em ambos os DataFrames seja do tipo int
dados_agrupados_pandas['ano'] = dados_agrupados_pandas['ano'].astype(int)
projecao_nacional_pandas['ano'] = projecao_nacional_pandas['ano'].astype(int)

In [0]:
# Calcular a taxa de internação e armazenar na coluna 'total_internacoes'
dados_agrupados_pandas = dados_agrupados_pandas.merge(projecao_nacional_pandas, on='ano', how='left')
dados_agrupados_pandas['internacoes'] = dados_agrupados_pandas['total_internacoes']
dados_agrupados_pandas['total_internacoes'] = round((dados_agrupados_pandas['internacoes']/dados_agrupados_pandas['populacao'])*1000000000, 2)

In [0]:
# Ajustando dados por UF
sih_pandas['DT_INTER'] = pd.to_datetime(sih_pandas['DT_INTER'], errors='coerce')

internacoes_uf = sih_pandas.groupby(['DT_INTER', 'UF_ZI', 'DIAG_PRINC_NOME'])['N_AIH'].nunique().reset_index()
internacoes_uf.rename(columns={'N_AIH': 'INTERNACOES'}, inplace=True)
internacoes_uf['UF_ZI'] = internacoes_uf['UF_ZI'].astype(str).str[:2]
internacoes_uf['ANO'] = internacoes_uf['DT_INTER'].dt.year

In [0]:
# Ajustar dados populacionais
projecao_long = projecao_populacional_pandas.melt(id_vars=['geocod', 'unidade_federada'],
                                                  var_name='ano',
                                                  value_name='populacao')
projecao_long['ano'] = projecao_long['ano'].str.extract('(\d{4})')

In [0]:
# Garantir que ANO e ano sejam inteiros antes de filtrar
internacoes_uf['ANO'] = internacoes_uf['ANO'].astype(int)
projecao_long['ano'] = projecao_long['ano'].astype(int)

# Remover espaços e extrair apenas os dígitos em UF_ZI e geocod
internacoes_uf['UF_ZI'] = internacoes_uf['UF_ZI'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(2)
projecao_long['geocod'] = projecao_long['geocod'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(2)

# Filtrar o dataframe internacoes_uf para anos entre 2018 e 2024
internacoes_uf = internacoes_uf[(internacoes_uf['ANO'] >= 2018) & (internacoes_uf['ANO'] <= 2024)]

# Filtrar o dataframe projecao_long para anos entre 2018 e 2024
projecao_long = projecao_long[(projecao_long['ano'] >= 2018) & (projecao_long['ano'] <= 2024)]

# Garantir que ANO e ano sejam inteiros
internacoes_uf['ANO'] = internacoes_uf['ANO'].astype(int)
projecao_long['ano'] = projecao_long['ano'].astype(int)


In [0]:
# Dados de UF 
dados_agrupados_pandas_uf = pd.merge(internacoes_uf, projecao_long, left_on=['UF_ZI', 'ANO'], right_on=['geocod', 'ano'], how='left')
dados_agrupados_pandas_uf.drop(columns=['geocod', 'ano'], inplace=True)

In [0]:
# Calculando Taxa de Internação por dia
dados_agrupados_pandas_uf['taxa_internacao'] = round(dados_agrupados_pandas_uf['INTERNACOES']/dados_agrupados_pandas_uf['populacao']*1000000, 2)

In [0]:
# Dicionário para mapear estados para suas siglas
estado_para_sigla = {
    'Rondônia': 'RO',
    'Acre': 'AC',
    'Amazonas': 'AM',
    'Roraima': 'RR',
    'Pará': 'PA',
    'Amapá': 'AP',
    'Tocantins': 'TO',
    'Maranhão': 'MA',
    'Piauí': 'PI',
    'Ceará': 'CE',
    'Rio Grande do Norte': 'RN',
    'Paraíba': 'PB',
    'Pernambuco': 'PE',
    'Alagoas': 'AL',
    'Sergipe': 'SE',
    'Bahia': 'BA',
    'Minas Gerais': 'MG',
    'Espírito Santo': 'ES',
    'Rio de Janeiro': 'RJ',
    'São Paulo': 'SP',
    'Paraná': 'PR',
    'Santa Catarina': 'SC',
    'Rio Grande do Sul': 'RS',
    'Mato Grosso do Sul': 'MS',
    'Mato Grosso': 'MT',
    'Goiás': 'GO',
    'Distrito Federal': 'DF'
}

# Substituir os nomes dos estados pelas siglas
dados_agrupados_pandas_uf['unidade_federada'] = dados_agrupados_pandas_uf['unidade_federada'].map(estado_para_sigla)

In [0]:
# Substituir DIAG_PRINC_NOME por DIAG_PRINC em dados_agrupados_pandas_uf
dados_agrupados_pandas_uf = dados_agrupados_pandas_uf.merge(
    correspondencia_pandas_cid[['DIAG_PRINC', 'DIAG_PRINC_NOME']],
    on='DIAG_PRINC_NOME',
    how='left'
)

# Substituir content_main_theme por DIAG_PRINC em wb_micro_pandas_uf_group
wb_micro_pandas_uf_group = wb_micro_pandas_uf_group.merge(
    correspondencia_pandas_cid[['DIAG_PRINC', 'CATEGORIA_PRIMARIA']],
    left_on='content_main_theme',
    right_on='CATEGORIA_PRIMARIA',
    how='left'
)

In [0]:
# Agora vamos garantir que as colunas de data estejam no formato correto
dados_agrupados_pandas_uf['DT_INTER'] = pd.to_datetime(dados_agrupados_pandas_uf['DT_INTER'], errors='coerce')
wb_micro_pandas_uf_group['access_date'] = pd.to_datetime(wb_micro_pandas_uf_group['access_date'], errors='coerce')

# Realizar o merge com left join usando as colunas ajustadas
merged_uf_df = pd.merge(
    dados_agrupados_pandas_uf,
    wb_micro_pandas_uf_group,
    left_on=['DT_INTER', 'DIAG_PRINC', 'unidade_federada'],
    right_on=['access_date', 'DIAG_PRINC', 'access_state'],
    how='left'
)

# Apagar as colunas excedentes do dataframe após o merge
merged_uf_df.drop(columns=['access_date', 'access_state', 'content_main_theme', 'DIAG_PRINC_NOME', 'CATEGORIA_PRIMARIA'], inplace=True)

# Preencher valores ausentes na coluna 'count' com zero
merged_uf_df['count'] = merged_uf_df['count'].fillna(0)

In [0]:
# Remover linhas onde DIAG_PRINC é NaN
merged_uf_df = merged_uf_df[merged_uf_df['DIAG_PRINC'].notna()]

# Filtrar as linhas para datas entre 2021-01-01 e 2024-06-01
merged_uf_df = merged_uf_df[
    (merged_uf_df['DT_INTER'] >= '2021-01-01') & 
    (merged_uf_df['DT_INTER'] <= '2024-05-30')
]

# Unindo com dados de correspondência de cid
merged_uf_df = pd.DataFrame(merged_uf_df.merge(correspondencia_pandas_cid, on = "DIAG_PRINC", how = "left"))

# Fazendo cópia do df original
merged_uf_df_copy = merged_uf_df.copy()

In [0]:
# Agrupando dados por categoria primária
merged_uf_df = pd.DataFrame(merged_uf_df.groupby(['DT_INTER', 'unidade_federada', 'CATEGORIA_PRIMARIA']).agg({'INTERNACOES':'sum', 'count':'sum', 'populacao':'max'})).reset_index()

# Calculando taxa de internação
merged_uf_df['taxa_internacao'] = round(merged_uf_df['INTERNACOES']/merged_uf_df['populacao']*100000, 2)

### Descrição por território

In [0]:
# Soma da coluna INTERNACOES por unidade_federada e ano
tabela_uf_ano_frequencia = (
    merged_uf_df.assign(ANO=pd.to_datetime(merged_uf_df["DT_INTER"]).dt.year)
    .groupby(["unidade_federada", "ANO"])["INTERNACOES"]
    .sum()
    .reset_index()
    .pivot(index="unidade_federada", columns="ANO", values="INTERNACOES")
)

# Soma da coluna taxa_internacao por unidade_federada e ano
tabela_uf_ano_taxa = (
    merged_uf_df.assign(ANO=pd.to_datetime(merged_uf_df["DT_INTER"]).dt.year)
    .groupby(["unidade_federada", "ANO"])["taxa_internacao"]
    .sum()
    .reset_index()
    .pivot(index="unidade_federada", columns="ANO", values="taxa_internacao")
)

In [0]:
# Salvar tabelas localmente
tabela_uf_ano_frequencia.to_excel("/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_descritiva_sih/dados_demograficos/tabela_uf_ano_frequencia.xlsx")
tabela_uf_ano_taxa.to_excel("/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_descritiva_sih/dados_demograficos/tabela_uf_ano_taxa.xlsx")

In [0]:
from geobr import read_state
import matplotlib.pyplot as plt
import pandas as pd

# Carregar shapefile das unidades federativas do Brasil usando geobr
brasil_shapefile = read_state(year=2020)

# Garantir que as siglas no shapefile correspondam ao índice da tabela_uf_ano_taxa
brasil_shapefile["SIGLA"] = brasil_shapefile["abbrev_state"]

In [0]:
# Ajuste o caminho de salvamento para o seu ambiente local
for ano in tabela_uf_ano_taxa.columns:
    # Mesclar dados da tabela com o shapefile
    mapa_dados = brasil_shapefile.merge(
        tabela_uf_ano_taxa[[ano]].reset_index(), how="left", left_on="SIGLA", right_on="unidade_federada"
    )
    
    # Calcular os quartis para classificação
    mapa_dados["quartil"] = pd.qcut(mapa_dados[ano], q=4, labels=["Q1", "Q2", "Q3", "Q4"])
    
    # Criar o mapa
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    mapa_dados.plot(
        column=ano,
        cmap="Blues",
        legend=True,
        legend_kwds={
            "label": f"Taxa de Internação ({ano})",
            "orientation": "horizontal",
            "shrink": 0.7,  # Reduzir a barra da legenda
        },
        edgecolor="black",  # Borda preta fina
        linewidth=0.5,
        ax=ax,
        missing_kwds={"color": "lightgrey", "label": "Sem Dados"},
    )
    
    # Configurações do mapa
    ax.set_title(f"Taxa de Internação por Unidade Federativa ({ano})", fontsize=16)
    ax.axis("off")
    
    # Adicionar rosa dos ventos no canto inferior direito
    ax.arrow(0.9, 0.1, 0, 0.05, transform=ax.transAxes, color="black",
             head_width=0.02, head_length=0.03, length_includes_head=True)
    ax.text(0.92, 0.12, "N", transform=ax.transAxes, fontsize=12, weight="bold")
    
    # Adicionar escala no canto inferior direito
    scalebar = ScaleBar(100, location="lower right", length_fraction=0.1)
    ax.add_artist(scalebar)
    
    plt.tight_layout()
    
    # Salvar o mapa localmente
    plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_descritiva_sih/graficos/mapa_taxa_internacao_ajustado_{ano}.png", dpi=300)
    plt.close()


### Descrição dos dados de acessos e internações

In [0]:
# Criar gráficos de linha para cada tema_principal
for tema in dados_agrupados_pandas['tema_principal'].unique():
    # Filtrar os dados para o diagnóstico atual
    dados_tema = dados_agrupados_pandas[dados_agrupados_pandas['tema_principal'] == tema].copy()

    # Ajustando range de datas
    datas_range = pd.DataFrame({
        "DT_INTER": pd.date_range(start="2021-01-01", end="2024-01-01")
    })

    # Unindo dados com datas
    dados_tema["DT_INTER"] = pd.to_datetime(dados_tema["DT_INTER"])
    dados_tema = pd.merge(datas_range, dados_tema, on="DT_INTER", how="left")

    # Substituir NaN por 0 em colunas numéricas
    num_cols = ["total_acessos", "total_internacoes", "internacoes"]
    for col in num_cols:
        if col in dados_tema.columns:  # Verificar se a coluna existe
            dados_tema[col] = dados_tema[col].fillna(0)

    # Ajustando valores não numéricos com ffill e bfill
    non_num_cols = ["unidade_federada", "CATEGORIA_PRIMARIA", "populacao"]
    for col in non_num_cols:
        if col in dados_tema.columns:  # Verificar se a coluna existe
            dados_tema[col] = dados_tema[col].ffill().bfill()
    
    # Salvando em excel
    #-- dados_tema.to_excel(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/dados_diarios/{tema}_{correlacao}_series_temporais_diario.xlsx")
    
    # Calcular a correlação entre 'total_acessos' e 'total_internacoes'
    correlacao = round(dados_tema[['internacoes', 'total_acessos']].corr().iloc[0, 1], 2)
    
    # Plotar as séries temporais
    plt.figure(figsize=(10,6))
    
    # Série temporal de 'total_acessos'
    plt.plot(dados_tema['DT_INTER'], dados_tema['total_acessos'], label='Total de acessos no Whitebook', color='blue')
    
    # Série temporal de 'total_internacoes'
    plt.plot(dados_tema['DT_INTER'], dados_tema['internacoes'], label='Total de internações no SIH', color='green')
    
    # Adicionar título e rótulos com a correlação
    plt.title(f'Série Temporal- {tema} (Correlação: {correlacao:.2f})')
    plt.xlabel('Data')
    plt.ylabel('Quantidade')
    plt.legend()
    
    # Ajuste de layout
    plt.xticks(rotation=90)
    plt.tight_layout()
    
    # Salvar o gráfico
    #-- plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/graficos_correlacao/{tema}_{correlacao}_series_temporais.png")
    
    # Exibir o gráfico no notebook
    plt.show()

In [0]:
# Lista para acumular os resultados
resultados_correlacao = []

# Criar gráficos de linha para cada tema_principal e unidade_federada
for tema in merged_uf_df['CATEGORIA_PRIMARIA'].unique():
    for uf in merged_uf_df['unidade_federada'].unique():
        # Filtrar os dados para o diagnóstico atual
        dados_tema = merged_uf_df[
            (merged_uf_df['CATEGORIA_PRIMARIA'] == tema) & (merged_uf_df['unidade_federada'] == uf)
        ].copy()

        # Ajustando range de datas
        datas_range = pd.DataFrame({
            "DT_INTER": pd.date_range(start="2021-01-01", end="2024-05-30")
        })

        # Unindo dados com datas
        dados_tema["DT_INTER"] = pd.to_datetime(dados_tema["DT_INTER"])
        dados_tema = pd.merge(datas_range, dados_tema, on="DT_INTER", how="left")

        # Substituir NaN por 0 em colunas numéricas
        num_cols = ["INTERNACOES", "count", "taxa_internacao"]
        for col in num_cols:
            if col in dados_tema.columns:  # Verificar se a coluna existe
                dados_tema[col] = dados_tema[col].fillna(0)

        # Ajustando valores não numéricos com ffill e bfill
        non_num_cols = ["unidade_federada", "CATEGORIA_PRIMARIA", "populacao"]
        for col in non_num_cols:
            if col in dados_tema.columns:  # Verificar se a coluna existe
                dados_tema[col] = dados_tema[col].ffill().bfill()
        
        # Calcular a correlação entre 'INTERNACOES' e 'count'
        correlacao = round(dados_tema[['INTERNACOES', 'count']].corr().iloc[0, 1], 2)

        # Adicionar o resultado à lista
        resultados_correlacao.append({"uf": uf, "tema": tema, "correlacao": correlacao})

        # Salvar o DataFrame processado em Excel
        caminho_excel = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/dados_diarios/{uf}_{tema}_{correlacao}_series_temporais_diario.xlsx"
        #-- dados_tema.to_excel(caminho_excel, index=False)

        # Plotar as séries temporais
        plt.figure(figsize=(10, 6))
        plt.plot(dados_tema['DT_INTER'], dados_tema['count'], label='Total de acessos no Whitebook', color='blue')
        plt.plot(dados_tema['DT_INTER'], dados_tema['INTERNACOES'], label='Total de internações no SIH', color='green')

        # Adicionar título e rótulos com a correlação
        plt.title(f'Série Temporal - {tema} em {uf}, (Correlação: {correlacao:.2f})')
        plt.xlabel('Data')
        plt.ylabel('Quantidade')
        plt.legend()
        plt.xticks(rotation=90)
        plt.tight_layout()

        # Salvar o gráfico diretamente
        caminho_grafico = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/graficos_correlacao/{uf}_{tema}_{correlacao}_series_temporais.png"
        #-- plt.savefig(caminho_grafico, dpi=300)

        # Fechar o gráfico para liberar memória
        plt.close()

        # Limpar o DataFrame processado
        del dados_tema

In [0]:
# Criar o DataFrame de resultados de correlação
df_resultados = pd.DataFrame(resultados_correlacao)

# Ordenar pela correlação (maior para menor)
df_resultados = df_resultados.sort_values(by="correlacao", ascending=False)

# Salvar o DataFrame final em um arquivo Excel
caminho_final = "/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/graficos_correlacao/resultados_correlacao.xlsx"

df_resultados.to_excel(caminho_final, index=False)

## Treinamento de modelos de Séries Temporais

### Modelos nacionais

In [0]:
# Função para dividir os dados em treino (antes de 2023) e teste (2023)
def prepare_data(df):
    # Converter '2023-01-01' para um objeto datetime compatível com datetime64[ns]
    data_limite = pd.to_datetime('2023-01-01')

    # Como 'DT_INTER' virou índice, usamos o índice para a filtragem
    treino = df[df.index < data_limite]
    teste = df[df.index >= data_limite]
    
    return treino, teste

In [0]:
def prepare_time_series_data(df, tema):
    # Filtrar o tema principal específico
    dados_tema = df[df['tema_principal'] == tema].copy()

    # Selecionar as colunas necessárias
    dados_tema = dados_tema[['DT_INTER', 'internacoes', 'total_acessos']]

    # Garantir que DT_INTER seja tratado como data
    dados_tema['DT_INTER'] = pd.to_datetime(dados_tema['DT_INTER'])

    # Definir DT_INTER como índice
    dados_tema.set_index('DT_INTER', inplace=True)

    # Verificar o intervalo completo de datas desde o início até o fim
    all_dates = pd.date_range(start=dados_tema.index.min(), end=dados_tema.index.max())

    # Adicionar as datas faltantes e preencher com NaN
    dados_tema = dados_tema.reindex(all_dates, fill_value=pd.NA)

    # Interpolar os dados para preencher os valores NaN
    dados_tema.interpolate(method='time', inplace=True)

    # Remover as lacunas com mais de 3 dias consecutivos sem dados
    # Criar uma série booleana que identifica onde estão os NaNs
    is_nan = dados_tema['internacoes'].isna() | dados_tema['total_acessos'].isna()

    # Agrupar os NaNs e contar os grupos consecutivos
    grouped_nan = is_nan.groupby((is_nan != is_nan.shift()).cumsum()).cumsum()

    # Encontrar os índices onde a contagem de NaNs consecutivos é maior que 3
    to_remove = grouped_nan[grouped_nan > 3].index

    # Remover esses índices do DataFrame
    dados_tema.drop(index=to_remove, inplace=True)

    return dados_tema

#### Modelo Univariado

In [0]:
# Função para treinar o modelo SARIMAX com ou sem variável exógena
def treinar_modelo_sarimax_univariado(treino, teste):
    # Separar a variável dependente (internacoes)
    y_train = treino['internacoes']
    y_test = teste['internacoes']
    
    # Criar e ajustar o modelo SARIMAX (com ou sem variável exógena)
    modelo = SARIMAX(y_train, exog=None, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit(disp=False)
    
    # Fazer previsões para o período de teste
    forecast = resultado.get_forecast(steps=len(teste))
    
    # Gerar o intervalo de confiança
    conf_int = forecast.conf_int(alpha=0.05)
    
    return resultado, forecast, conf_int

In [0]:
# Função para plotar as previsões e dados reais
def plot_forecast_univariado(treino, teste, forecast, conf_int, tema):
    plt.figure(figsize=(10,6))
    
    # Plotar os dados de treino
    plt.plot(treino.index, treino['internacoes'], label='Treino (Observado)', color='blue')
    
    # Plotar os dados de teste
    plt.plot(teste.index, teste['internacoes'], label='Teste (Observado)', color='orange')
    
    # Plotar o forecast
    plt.plot(teste.index, forecast.predicted_mean, label='Forecast (Predição)', color='green')
    
    # Plotar intervalo de confiança
    plt.fill_between(teste.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='gray', alpha=0.3, label='95% Intervalo de Confiança')
    
    # Configurações do gráfico
    plt.title(f'Forecast SARIMAX univariado - {tema}')
    plt.xlabel('Data')
    plt.ylabel('Total Internações')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    
    # Salvar o gráfico com o prefixo "ts_", tema e tipo de modelo
    plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/ts_univariada/graficos/ts_{tema}_univariado_forecast_nacional.png")
    plt.close()

In [0]:
# Função para gerar e salvar o sumário do modelo em Excel
def summary_modelo_univariado(resultado, tema):
    print(f'Summary do modelo SARIMAX univariado para {tema}')
    print(resultado.summary())
    
    # Converte o summary em uma string e divide em linhas
    summary_text = resultado.summary().as_text().splitlines()
    
    # Converte cada linha do summary em uma lista de strings para formar um DataFrame
    summary_df = pd.DataFrame([line.split() for line in summary_text if line.strip()], dtype=str)
    
    # Define o caminho do arquivo de saída
    output_path = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/ts_univariada/summaries/ts_{tema}_univariado_summary.xlsx"
    
    # Salva o DataFrame no Excel
    summary_df.to_excel(output_path, index=False, header=False)
    
    print(f"Summary completo salvo em: {output_path}")

In [0]:
# Loop para cada tema principal
for tema in dados_agrupados_pandas['tema_principal'].unique():
    try:
        # Preparar os dados de série temporal
        dados_tema = prepare_time_series_data(dados_agrupados_pandas, tema)
        
        # Dividir em treino (anos antes de 2023) e teste (2023)
        treino, teste = prepare_data(dados_tema)
        
        # Treinar e salvar o modelo univariado (sem exógena)
        resultado_uni, forecast_uni, conf_int_uni = treinar_modelo_sarimax_univariado(treino, teste)
        plot_forecast_univariado(treino, teste, forecast_uni, conf_int_uni, tema)
        summary_modelo_univariado(resultado_uni, tema)
            
    except ValueError as ve:
        # Se ocorrer um ValueError (como o array vazio), imprimir a mensagem de erro
        print(f"Erro ao processar o tema '{tema}': {ve}")
    
    except Exception as e:
        # Capturar outros tipos de exceções e imprimir a mensagem de erro
        print(f"Ocorreu um erro inesperado ao processar o tema '{tema}': {e}")

#### Modelo com variável exógena (Whitebook)

In [0]:
# Função para treinar o modelo SARIMAX com ou sem variável exógena
def treinar_modelo_sarimax_exogena(treino, teste, com_exog=True):
    # Separar a variável dependente (internacoes)
    y_train = treino['internacoes']
    y_test = teste['internacoes']
    
    exog_train = treino[['total_acessos']]
    exog_test = teste[['total_acessos']]
   
    # Criar e ajustar o modelo SARIMAX (com ou sem variável exógena)
    modelo = SARIMAX(y_train, exog=exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit(disp=False)
    
    # Fazer previsões para o período de teste
    forecast = resultado.get_forecast(steps=len(teste), exog=exog_test)
    
    # Gerar o intervalo de confiança
    conf_int = forecast.conf_int(alpha=0.05)
    
    return resultado, forecast, conf_int

In [0]:
# Função para plotar as previsões e dados reais
def plot_forecast_exogena(treino, teste, forecast, conf_int, tema):
    plt.figure(figsize=(10,6))
    
    # Plotar os dados de treino
    plt.plot(treino.index, treino['internacoes'], label='Treino (Observado)', color='blue')
    
    # Plotar os dados de teste
    plt.plot(teste.index, teste['internacoes'], label='Teste (Observado)', color='orange')
    
    # Plotar o forecast
    plt.plot(teste.index, forecast.predicted_mean, label='Forecast (Predição)', color='green')
    
    # Plotar intervalo de confiança
    plt.fill_between(teste.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='gray', alpha=0.3, label='95% Intervalo de Confiança')
    
    # Configurações do gráfico
    plt.title(f'Forecast SARIMAX com variável exógena - {tema}')
    plt.xlabel('Data')
    plt.ylabel('Total Internações')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    
    # Salvar o gráfico com o prefixo "ts_", tema e tipo de modelo
    plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/ts_exogena/graficos/ts_{tema}_exogena_forecast_nacional.png")
    plt.close()

In [0]:
# Função para gerar e salvar o sumário do modelo em Excel
def summary_modelo_exogena(resultado, tema):
    print(f'Summary do modelo SARIMAX com variável exógena para {tema}')
    print(resultado.summary())
    
    # Converte o summary em uma string e divide em linhas
    summary_text = resultado.summary().as_text().splitlines()
    
    # Converte cada linha do summary em uma lista de strings para formar um DataFrame
    summary_df = pd.DataFrame([line.split() for line in summary_text if line.strip()], dtype=str)
    
    # Define o caminho do arquivo de saída
    output_path = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_nacional/ts_exogena/summaries/ts_{tema}_exaogena_summary.xlsx"
    
    # Salva o DataFrame no Excel
    summary_df.to_excel(output_path, index=False, header=False)
    
    print(f"Summary completo salvo em: {output_path}")

In [0]:
# Loop para cada tema principal
for tema in dados_agrupados_pandas['tema_principal'].unique():
    try:
        # Preparar os dados de série temporal
        dados_tema = prepare_time_series_data(dados_agrupados_pandas, tema)
        
        # Dividir em treino (anos antes de 2023) e teste (2023)
        treino, teste = prepare_data(dados_tema)
                
        # Treinar e salvar o modelo com exógena
        resultado_exog, forecast_exog, conf_int_exog = treinar_modelo_sarimax_exogena(treino, teste)
        plot_forecast_exogena(treino, teste, forecast_exog, conf_int_exog, tema)
        summary_modelo_exogena(resultado_exog, tema)
    
    except ValueError as ve:
        # Se ocorrer um ValueError (como o array vazio), imprimir a mensagem de erro
        print(f"Erro ao processar o tema '{tema}': {ve}")
    
    except Exception as e:
        # Capturar outros tipos de exceções e imprimir a mensagem de erro
        print(f"Ocorreu um erro inesperado ao processar o tema '{tema}': {e}")

### Modelos estaduais

In [0]:
def prepare_time_series_data_uf(df, tema, uf):
    # Filtrar o tema principal específico
    dados_tema = df[(df['tema_principal'] == tema) & (df['UF'] == uf)].copy()

    # Selecionar as colunas necessárias
    dados_tema = dados_tema[['DT_INTER', 'internacoes', 'total_acessos']]

    # Garantir que DT_INTER seja tratado como data
    dados_tema['DT_INTER'] = pd.to_datetime(dados_tema['DT_INTER'])

    # Definir DT_INTER como índice
    dados_tema.set_index('DT_INTER', inplace=True)

    # Verificar o intervalo completo de datas desde o início até o fim
    all_dates = pd.date_range(start=dados_tema.index.min(), end=dados_tema.index.max())

    # Adicionar as datas faltantes e preencher com NaN
    dados_tema = dados_tema.reindex(all_dates, fill_value=pd.NA)

    # Interpolar os dados para preencher os valores NaN
    dados_tema.interpolate(method='time', inplace=True)

    # Remover as lacunas com mais de 3 dias consecutivos sem dados
    # Criar uma série booleana que identifica onde estão os NaNs
    is_nan = dados_tema['internacoes'].isna() | dados_tema['total_acessos'].isna()

    # Agrupar os NaNs e contar os grupos consecutivos
    grouped_nan = is_nan.groupby((is_nan != is_nan.shift()).cumsum()).cumsum()

    # Encontrar os índices onde a contagem de NaNs consecutivos é maior que 3
    to_remove = grouped_nan[grouped_nan > 3].index

    # Remover esses índices do DataFrame
    dados_tema.drop(index=to_remove, inplace=True)

    return dados_tema

#### Modelo Univariado

In [0]:
# Ajustando nome das colunas
dados_agrupados_pandas_uf = merged_uf_df.copy()
dados_agrupados_pandas_uf.columns = ("DT_INTER", "UF", "tema_principal", "internacoes", "total_acessos", "populacao", "taxa_internacao")

In [0]:
# Função para treinar o modelo SARIMAX com ou sem variável exógena
def treinar_modelo_sarimax_univariado_uf(treino, teste):
    # Separar a variável dependente (internacoes)
    y_train = treino['internacoes']
    y_test = teste['internacoes']
    
    # Criar e ajustar o modelo SARIMAX (com ou sem variável exógena)
    modelo = SARIMAX(y_train, exog=None, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit(disp=False)
    
    # Fazer previsões para o período de teste
    forecast = resultado.get_forecast(steps=len(teste))
    
    # Gerar o intervalo de confiança
    conf_int = forecast.conf_int(alpha=0.05)
    
    return resultado, forecast, conf_int

In [0]:
# Função para plotar as previsões e dados reais
def plot_forecast_univariado_uf(treino, teste, forecast, conf_int, tema):
    plt.figure(figsize=(10,6))
    
    # Plotar os dados de treino
    plt.plot(treino.index, treino['internacoes'], label='Treino (Observado)', color='blue')
    
    # Plotar os dados de teste
    plt.plot(teste.index, teste['internacoes'], label='Teste (Observado)', color='orange')
    
    # Plotar o forecast
    plt.plot(teste.index, forecast.predicted_mean, label='Forecast (Predição)', color='green')
    
    # Plotar intervalo de confiança
    plt.fill_between(teste.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='gray', alpha=0.3, label='95% Intervalo de Confiança')
    
    # Configurações do gráfico
    plt.title(f'Forecast SARIMAX univariado - {tema} em {uf}')
    plt.xlabel('Data')
    plt.ylabel('Total Internações')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    
    # Salvar o gráfico com o prefixo "ts_", tema e tipo de modelo
    plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/ts_univariada/graficos/ts_{tema}_{uf}_univariado_forecast.png")
    plt.close()

In [0]:
# Função para gerar e salvar o sumário do modelo em Excel
def summary_modelo_univariado_uf(resultado, tema):
    print(f'Summary do modelo SARIMAX univariado para {tema}')
    print(resultado.summary())
    
    # Converte o summary em uma string e divide em linhas
    summary_text = resultado.summary().as_text().splitlines()
    
    # Converte cada linha do summary em uma lista de strings para formar um DataFrame
    summary_df = pd.DataFrame([line.split() for line in summary_text if line.strip()], dtype=str)
    
    # Define o caminho do arquivo de saída
    output_path = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/ts_univariada/summaries/ts_{tema}_{uf}_univariado.xlsx"
    
    # Salva o DataFrame no Excel
    summary_df.to_excel(output_path, index=False, header=False)
    
    print(f"Summary completo salvo em: {output_path}")

In [0]:
# Loop para cada tema principal
for tema in dados_agrupados_pandas_uf['tema_principal'].unique():
    for uf in dados_agrupados_pandas_uf['UF'].unique():
        try:
            # Preparar os dados de série temporal
            dados_tema = prepare_time_series_data_uf(dados_agrupados_pandas_uf, tema, uf)
            
            # Dividir em treino (anos antes de 2023) e teste (2023)
            treino, teste = prepare_data(dados_tema)
            
            # Treinar e salvar o modelo univariado (sem exógena)
            resultado_uni, forecast_uni, conf_int_uni = treinar_modelo_sarimax_univariado_uf(treino, teste)
            plot_forecast_univariado_uf(treino, teste, forecast_uni, conf_int_uni, tema)
            summary_modelo_univariado_uf(resultado_uni, tema)
                
        except ValueError as ve:
            # Se ocorrer um ValueError (como o array vazio), imprimir a mensagem de erro
            print(f"Erro ao processar o tema '{tema}' na {uf}: {ve}")
        
        except Exception as e:
            # Capturar outros tipos de exceções e imprimir a mensagem de erro
            print(f"Ocorreu um erro inesperado ao processar o tema '{tema}' na {uf}: {e}")

#### Modelo com variável exógena (Whitebook)

In [0]:
# Função para treinar o modelo SARIMAX com ou sem variável exógena
def treinar_modelo_sarimax_exogena_uf(treino, teste, com_exog=True):
    # Separar a variável dependente (internacoes)
    y_train = treino['internacoes']
    y_test = teste['internacoes']
    
    exog_train = treino[['total_acessos']]
    exog_test = teste[['total_acessos']]
   
    # Criar e ajustar o modelo SARIMAX (com ou sem variável exógena)
    modelo = SARIMAX(y_train, exog=exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit(disp=False)
    
    # Fazer previsões para o período de teste
    forecast = resultado.get_forecast(steps=len(teste), exog=exog_test)
    
    # Gerar o intervalo de confiança
    conf_int = forecast.conf_int(alpha=0.05)
    
    return resultado, forecast, conf_int

In [0]:
# Função para plotar as previsões e dados reais
def plot_forecast_exogena_uf(treino, teste, forecast, conf_int, tema):
    plt.figure(figsize=(10,6))
    
    # Plotar os dados de treino
    plt.plot(treino.index, treino['internacoes'], label='Treino (Observado)', color='blue')
    
    # Plotar os dados de teste
    plt.plot(teste.index, teste['internacoes'], label='Teste (Observado)', color='orange')
    
    # Plotar o forecast
    plt.plot(teste.index, forecast.predicted_mean, label='Forecast (Predição)', color='green')
    
    # Plotar intervalo de confiança
    plt.fill_between(teste.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='gray', alpha=0.3, label='95% Intervalo de Confiança')
    
    # Configurações do gráfico
    plt.title(f'Forecast SARIMAX com variável exógena - {tema} em {uf}')
    plt.xlabel('Data')
    plt.ylabel('Total Internações')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    
    # Salvar o gráfico com o prefixo "ts_", tema e tipo de modelo
    plt.savefig(f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/ts_exogena/graficos/ts_{tema}_{uf}_exogena_forecast.png")
    plt.close()

In [0]:
# Função para gerar e salvar o sumário do modelo em Excel
def summary_modelo_exogena_uf(resultado, tema):
    print(f'Summary do modelo SARIMAX com variável exógena para {tema} em {uf}')
    print(resultado.summary())
    
    # Converte o summary em uma string e divide em linhas
    summary_text = resultado.summary().as_text().splitlines()
    
    # Converte cada linha do summary em uma lista de strings para formar um DataFrame
    summary_df = pd.DataFrame([line.split() for line in summary_text if line.strip()], dtype=str)
    
    # Define o caminho do arquivo de saída
    output_path = f"/Workspace/Users/danielly.xavier@iclinic.com.br/resultados_modelo_ts_research_center/analise_estadual/ts_exogena/summaries/ts_{tema}_{uf}_exaogena_summary.xlsx"
    
    # Salva o DataFrame no Excel
    summary_df.to_excel(output_path, index=False, header=False)
    
    print(f"Summary completo salvo em: {output_path}")

In [0]:
# Loop para cada tema principal
for tema in dados_agrupados_pandas_uf['tema_principal'].unique():
    for uf in dados_agrupados_pandas_uf['UF'].unique():
        try:
            # Preparar os dados de série temporal
            dados_tema = prepare_time_series_data_uf(dados_agrupados_pandas_uf, tema, uf)
            
            # Dividir em treino (anos antes de 2023) e teste (2023)
            treino, teste = prepare_data(dados_tema)
            
            # Treinar e salvar o modelo com variável exógena
            resultado_uni, forecast_uni, conf_int_uni = treinar_modelo_sarimax_exogena_uf(treino, teste)
            plot_forecast_exogena_uf(treino, teste, forecast_uni, conf_int_uni, tema)
            summary_modelo_exogena_uf(resultado_uni, tema)
                
        except ValueError as ve:
            # Se ocorrer um ValueError (como o array vazio), imprimir a mensagem de erro
            print(f"Erro ao processar o tema '{tema}' na {uf}: {ve}")
        
        except Exception as e:
            # Capturar outros tipos de exceções e imprimir a mensagem de erro
            print(f"Ocorreu um erro inesperado ao processar o tema '{tema}' na {uf}: {e}")

Extração de dados


In [0]:
import os
import pandas as pd

# Definir os caminhos dos arquivos
paths = {
    'univariado': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Asma_AC_univariado.xlsx',
    'exogena': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Asma_AC_exaogena_summary.xlsx'
}

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados
def extract_data(file_path, tipo):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)
        print(df.head(15))  # Visualização inicial

        prefix = 'UNIVARIADO' if tipo == 'univariado' else 'EXOGENA'

        # Extrair valores baseando-se na posição conhecida
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce') if tipo == 'exogena' else None
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce') if tipo == 'exogena' else None
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        uf = 'AC'
        agravo = 'Asma'
        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            f'LOG_{prefix}': log_likelihood,
            f'AIC_{prefix}': aic,
            f'BIC_{prefix}': bic,
            f'HQIC_{prefix}': hqic,
            f'LJUNGBOX_{prefix}': ljung_box,
            f'JAQUE_BERA_{prefix}': jarque_bera,
            f'KURTOSIS_{prefix}': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFIICATIVA': significancia if tipo == 'exogena' else None,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }
        print(f"Dados extraídos ({tipo}): {data}")

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar arquivos
for tipo, path in paths.items():
    extract_data(path, tipo)

# Consolidar resultados
df_resultados = pd.DataFrame(results_dict.values())

# Verificar valores ausentes
df_resultados['DADOS_INCOMPLETOS'] = df_resultados.isnull().any(axis=1)
df_resultados['CAMPOS_INCOMPLETOS'] = df_resultados.apply(
    lambda row: [col for col in df_resultados.columns if pd.isnull(row[col])], axis=1
)

# Exibir e salvar os resultados
print("\nResumo de campos incompletos:")
print(df_resultados[['UF', 'AGRAVO', 'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS']])
print("\nResultados consolidados:")
print(df_resultados)

output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_consolidados.csv'
df_resultados.to_csv(output_path, index=False)
print(f"Processamento concluído. Dados salvos em '{output_path}'.")


Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Asma_AC_univariado.xlsx
                                                   0           1   ...      9    10
0                                             SARIMAX     Results  ...     NaN  NaN
1                                                 NaN         NaN  ...     NaN  NaN
2                                                Dep.   Variable:  ...     NaN  NaN
3                                              Model:  SARIMAX(1,  ...  82.877  NaN
4                                               Date:        Tue,  ...     NaN  NaN
5                                               Time:    10:52:42  ...     NaN  NaN
6                                             Sample:  01-02-2021  ...     NaN  NaN
7                                                   -  12-31-2022  ...     NaN  NaN
8           

In [0]:
%python
%pip install openpyxl

import os
import pandas as pd
import glob

try:
    import openpyxl
except ImportError:
    print("O módulo 'openpyxl' não está instalado. Use 'pip install openpyxl' para instalá-lo.")
    raise

# Caminhos das pastas
paths = {
    'univariado': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries',
    'exogena': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries'
}

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados
def extract_data(file_path, tipo):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)
        
        # Verificar se o DataFrame está vazio
        if df.empty:
            print(f"Arquivo {file_path} está vazio ou não contém dados processáveis.")
            return
        
        print(df.head(15))  # Visualização inicial

        prefix = 'UNIVARIADO' if tipo == 'univariado' else 'EXOGENA'

        # Extrair valores baseando-se na posição conhecida
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce') if len(df) > 18 else None
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce') if len(df) > 18 else None
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce') if len(df) > 21 else None
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce') if tipo == 'exogena' else None
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce') if tipo == 'exogena' else None
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        uf = 'AC'
        agravo = 'Asma'
        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            f'LOG_{prefix}': log_likelihood,
            f'AIC_{prefix}': aic,
            f'BIC_{prefix}': bic,
            f'HQIC_{prefix}': hqic,
            f'LJUNGBOX_{prefix}': ljung_box,
            f'JAQUE_BERA_{prefix}': jarque_bera,
            f'KURTOSIS_{prefix}': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFIICATIVA': significancia if tipo == 'exogena' else None,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }
        print(f"Dados extraídos ({tipo}): {data}")

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os primeiros 5 arquivos de cada pasta
for tipo, path in paths.items():
    arquivos = glob.glob(os.path.join(path, "*.xlsx"))[:5]  # Selecionar os primeiros 5 arquivos
    if not arquivos:
        print(f"Nenhum arquivo encontrado na pasta: {path}")
    for file_path in arquivos:
        extract_data(file_path, tipo)

# Consolidar resultados
if results_dict:
    df_resultados = pd.DataFrame(results_dict.values())

    # Garantir que as colunas UF e AGRAVO existam
    if 'UF' not in df_resultados.columns:
        df_resultados['UF'] = None
    if 'AGRAVO' not in df_resultados.columns:
        df_resultados['AGRAVO'] = None

    # Verificar valores ausentes
    df_resultados['DADOS_INCOMPLETOS'] = df_resultados.isnull().any(axis=1)
    df_resultados['CAMPOS_INCOMPLETOS'] = df_resultados.apply(
        lambda row: [col for col in df_resultados.columns if pd.isnull(row[col])], axis=1
    )

    # Exibir e salvar os resultados
    print("\nResumo de campos incompletos:")
    print(df_resultados[['UF', 'AGRAVO', 'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS']])
    print("\nResultados consolidados:")
    print(df_resultados)

    # Salvar o resultado consolidado em um arquivo CSV
    output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_consolidados.csv'
    df_resultados.to_csv(output_path, index=False)
    print(f"Processamento concluído. Dados salvos em '{output_path}'.")
else:
    print("Nenhum dado foi extraído. Verifique os arquivos ou o processo de extração.")


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Pneumonia por outros organismos_RS_univariado.xlsx
                                                   0           1   ...   14   15
0                                             SARIMAX     Results  ...  NaN  NaN
1                                                 NaN         NaN  ...  NaN  NaN
2                                                Dep.   Variable:  ...  NaN  NaN
3                                              Model:  SARIMAX(1,  ...  NaN  NaN
4                                               Date:        Tue,  ...  NaN  NaN
5        

In [0]:
import os
import pandas as pd
import glob

# Definir os caminhos das pastas
paths = {
    'univariado': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/*.xlsx',
    'exogena': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'
}

# Dicionário para armazenar os resultados
results_dict = {}

# Função para extrair dados de uma planilha
def extract_data(file_path, tipo):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)
        print(df.head(15))  # Visualização inicial

        # Prefixo para diferenciar entre univariado e exógeno
        prefix = 'UNIVARIADO' if tipo == 'univariado' else 'EXOGENA'

        # Extrair valores
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce') if tipo == 'exogena' else None
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce') if tipo == 'exogena' else None
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        # Identificar UF e Agravo
        uf = file_path.split('_')[-2]
        agravo = file_path.split('_')[-3]
        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            f'LOG_{prefix}': log_likelihood,
            f'AIC_{prefix}': aic,
            f'BIC_{prefix}': bic,
            f'HQIC_{prefix}': hqic,
            f'LJUNGBOX_{prefix}': ljung_box,
            f'JAQUE_BERA_{prefix}': jarque_bera,
            f'KURTOSIS_{prefix}': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFIICATIVA': significancia if tipo == 'exogena' else None,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }
        print(f"Dados extraídos ({tipo}): {data}")

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os 5 primeiros arquivos de cada pasta
for tipo, path in paths.items():
    arquivos = glob.glob(path)[:5]  # Selecionar os primeiros 5 arquivos
    for file_path in arquivos:
        extract_data(file_path, tipo)

# Consolidar resultados em um DataFrame
df_resultados = pd.DataFrame(results_dict.values())

# Verificar colunas existentes no DataFrame resultante
colunas_disponiveis = df_resultados.columns

# Lista de colunas desejadas no formato final
colunas_ordenadas = [
    'UF', 'AGRAVO', 'LOG_UNIVARIADO', 'AIC_UNIVARIADO', 'BIC_UNIVARIADO', 'HQIC_UNIVARIADO',
    'LJUNGBOX_UNIVARIADO', 'JAQUE_BERA_UNIVARIADO', 'KURTOSIS_UNIVARIADO',
    'VARIAVEL_EXOGENA_SIGNIFIICATIVA', 'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE',
    'DESCRICAO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 'HQIC_EXOGENA',
    'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 'KURTOSIS_EXOGENA',
    'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS'
]

# Adicionar colunas ausentes com valores NaN
for coluna in colunas_ordenadas:
    if coluna not in colunas_disponiveis:
        df_resultados[coluna] = pd.NA

# Reorganizar colunas
df_resultados = df_resultados[colunas_ordenadas]

# Identificar valores ausentes
df_resultados['DADOS_INCOMPLETOS'] = df_resultados.isnull().any(axis=1)
df_resultados['CAMPOS_INCOMPLETOS'] = df_resultados.apply(
    lambda row: [col for col in df_resultados.columns if pd.isnull(row[col])], axis=1
)

# Salvar resultados consolidados
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_consolidados.csv'
df_resultados.to_csv(output_path, index=False)

# Exibir resumo
print("\nResumo de campos incompletos:")
print(df_resultados[['UF', 'AGRAVO', 'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS']])
print("\nResultados consolidados salvos em:")
print(output_path)


Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Pneumonia por outros organismos_RS_univariado.xlsx
                                                   0           1   ...   14   15
0                                             SARIMAX     Results  ...  NaN  NaN
1                                                 NaN         NaN  ...  NaN  NaN
2                                                Dep.   Variable:  ...  NaN  NaN
3                                              Model:  SARIMAX(1,  ...  NaN  NaN
4                                               Date:        Tue,  ...  NaN  NaN
5                                               Time:    11:43:00  ...  NaN  NaN
6                                             Sample:  09-18-2021  ...  NaN  NaN
7                                                   -  12-31-2022  ...  NaN  NaN
8           

In [0]:
import os
import pandas as pd
import glob

# Definir os caminhos das pastas
paths = {
    'univariado': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/*.xlsx',
    'exogena': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'
}

# Dicionário para armazenar os resultados
results_dict = {}

# Função para extrair dados de uma planilha
def extract_data(file_path, tipo):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)
        print(df.head(15))  # Visualização inicial

        # Prefixo para diferenciar entre univariado e exógeno
        prefix = 'UNIVARIADO' if tipo == 'univariado' else 'EXOGENA'

        # Extrair valores
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce') if tipo == 'exogena' else None
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce') if tipo == 'exogena' else None
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        # Identificar UF e Agravo
        uf = os.path.basename(file_path).split('_')[-2]
        agravo = os.path.basename(file_path).split('_')[-3]
        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            f'LOG_{prefix}': log_likelihood,
            f'AIC_{prefix}': aic,
            f'BIC_{prefix}': bic,
            f'HQIC_{prefix}': hqic,
            f'LJUNGBOX_{prefix}': ljung_box,
            f'JAQUE_BERA_{prefix}': jarque_bera,
            f'KURTOSIS_{prefix}': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFIICATIVA': significancia if tipo == 'exogena' else None,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {'UF': uf, 'AGRAVO': agravo}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os 5 primeiros arquivos de cada pasta
for tipo, path in paths.items():
    arquivos = glob.glob(path)[:5]  # Selecionar os primeiros 5 arquivos
    for file_path in arquivos:
        extract_data(file_path, tipo)

# Consolidar resultados em um DataFrame
df_resultados = pd.DataFrame(results_dict.values())

# Verificar colunas existentes no DataFrame resultante
colunas_disponiveis = df_resultados.columns

# Lista de colunas desejadas no formato final
colunas_ordenadas = [
    'UF', 'AGRAVO', 'LOG_UNIVARIADO', 'AIC_UNIVARIADO', 'BIC_UNIVARIADO', 'HQIC_UNIVARIADO',
    'LJUNGBOX_UNIVARIADO', 'JAQUE_BERA_UNIVARIADO', 'KURTOSIS_UNIVARIADO',
    'VARIAVEL_EXOGENA_SIGNIFIICATIVA', 'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE',
    'DESCRICAO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 'HQIC_EXOGENA',
    'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 'KURTOSIS_EXOGENA',
    'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS'
]

# Adicionar colunas ausentes com valores NaN
for coluna in colunas_ordenadas:
    if coluna not in colunas_disponiveis:
        df_resultados[coluna] = pd.NA

# Reorganizar colunas
df_resultados = df_resultados[colunas_ordenadas]

# Identificar valores ausentes
df_resultados['DADOS_INCOMPLETOS'] = df_resultados.isnull().any(axis=1)
df_resultados['CAMPOS_INCOMPLETOS'] = df_resultados.apply(
    lambda row: [col for col in df_resultados.columns if pd.isnull(row[col])], axis=1
)

# Salvar resultados consolidados
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_consolidados.csv'
df_resultados.to_csv(output_path, index=False)

# Exibir resumo
print("\nResumo de campos incompletos:")
print(df_resultados[['UF', 'AGRAVO', 'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS']])
print("\nResultados consolidados salvos em:")
print(output_path)


Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Pneumonia por outros organismos_RS_univariado.xlsx
                                                   0           1   ...   14   15
0                                             SARIMAX     Results  ...  NaN  NaN
1                                                 NaN         NaN  ...  NaN  NaN
2                                                Dep.   Variable:  ...  NaN  NaN
3                                              Model:  SARIMAX(1,  ...  NaN  NaN
4                                               Date:        Tue,  ...  NaN  NaN
5                                               Time:    11:43:00  ...  NaN  NaN
6                                             Sample:  09-18-2021  ...  NaN  NaN
7                                                   -  12-31-2022  ...  NaN  NaN
8           

In [0]:
import os
import pandas as pd
import glob

# Definir os caminhos das pastas
paths = {
    'univariado': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/*.xlsx',
    'exogena': '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'
}

# Dicionário para armazenar os resultados
results_dict = {}

# Função para extrair dados
def extract_data(file_path, tipo):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)
        
        # Prefixo para diferenciar entre univariado e exógeno
        prefix = 'UNIVARIADO' if tipo == 'univariado' else 'EXOGENA'

        # Extrair valores
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce') if tipo == 'exogena' else None
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce') if tipo == 'exogena' else None
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        # Identificar UF e Agravo
        filename = os.path.basename(file_path)
        parts = filename.replace(".xlsx", "").split('_')
        uf = parts[-2] if tipo == 'univariado' else parts[-3]
        agravo = parts[1]

        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            f'LOG_{prefix}': log_likelihood,
            f'AIC_{prefix}': aic,
            f'BIC_{prefix}': bic,
            f'HQIC_{prefix}': hqic,
            f'LJUNGBOX_{prefix}': ljung_box,
            f'JAQUE_BERA_{prefix}': jarque_bera,
            f'KURTOSIS_{prefix}': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFIICATIVA': significancia if tipo == 'exogena' else None,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Passo 1: Processar arquivos univariados
print("\n--- Processando UNIVARIADOS ---\n")
arquivos_univariados = glob.glob(paths['univariado'])[:5]
for file_path in arquivos_univariados:
    extract_data(file_path, 'univariado')

# Passo 2: Processar arquivos exógenos
print("\n--- Processando EXÓGENOS ---\n")
arquivos_exogenos = glob.glob(paths['exogena'])[:5]
for file_path in arquivos_exogenos:
    extract_data(file_path, 'exogena')

# Consolidar resultados em um DataFrame
df_resultados = pd.DataFrame(results_dict.values())

# Adicionar colunas ausentes
colunas_ordenadas = [
    'UF', 'AGRAVO', 'LOG_UNIVARIADO', 'AIC_UNIVARIADO', 'BIC_UNIVARIADO', 'HQIC_UNIVARIADO',
    'LJUNGBOX_UNIVARIADO', 'JAQUE_BERA_UNIVARIADO', 'KURTOSIS_UNIVARIADO',
    'VARIAVEL_EXOGENA_SIGNIFIICATIVA', 'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE',
    'DESCRICAO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 'HQIC_EXOGENA',
    'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 'KURTOSIS_EXOGENA',
    'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS'
]

for coluna in colunas_ordenadas:
    if coluna not in df_resultados.columns:
        df_resultados[coluna] = pd.NA

# Reorganizar as colunas
df_resultados = df_resultados[colunas_ordenadas]

# Identificar valores ausentes
df_resultados['DADOS_INCOMPLETOS'] = df_resultados.isnull().any(axis=1)
df_resultados['CAMPOS_INCOMPLETOS'] = df_resultados.apply(
    lambda row: [col for col in df_resultados.columns if pd.isnull(row[col])], axis=1
)

# Salvar resultados consolidados
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_consolidados.csv'
df_resultados.to_csv(output_path, index=False)

# Exibir resumo
print("\nResumo de campos incompletos:")
print(df_resultados[['UF', 'AGRAVO', 'DADOS_INCOMPLETOS', 'CAMPOS_INCOMPLETOS']])
print("\nResultados consolidados salvos em:")
print(output_path)



--- Processando UNIVARIADOS ---

Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Pneumonia por outros organismos_RS_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Covid-19_MT_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Cetoacidose diabética_RO_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/ts_Depressão_AL_u

Extração univariada


In [0]:
%python
%pip install openpyxl

import os
import pandas as pd
import glob

try:
    import openpyxl
except ImportError:
    print("O módulo 'openpyxl' não está instalado. Use 'pip install openpyxl' para instalá-lo.")
    raise


# Caminho da pasta com arquivos univariados
path_univariado = '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/*.xlsx'

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados
def extract_univariado_data(file_path):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)

        # Extrair valores baseando-se na posição conhecida
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[18, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[18, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')

        # Identificar UF e Agravo a partir do nome do arquivo
        filename = os.path.basename(file_path)
        parts = filename.replace(".xlsx", "").split('_')
        uf = parts[-2]
        agravo = " ".join(parts[1:-2])

        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            'LOG_UNIVARIADO': log_likelihood,
            'AIC_UNIVARIADO': aic,
            'BIC_UNIVARIADO': bic,
            'HQIC_UNIVARIADO': hqic,
            'LJUNGBOX_UNIVARIADO': ljung_box,
            'JAQUE_BERA_UNIVARIADO': jarque_bera,
            'KURTOSIS_UNIVARIADO': kurtosis,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os arquivos da pasta univariada
print("\n--- Processando arquivos UNIVARIADOS ---\n")
arquivos_univariados = glob.glob(path_univariado)
for file_path in arquivos_univariados:
    extract_univariado_data(file_path)

# Consolidar resultados
df_resultados_univariado = pd.DataFrame(results_dict.values())

# Verificar e preencher colunas ausentes
colunas_necessarias = [
    'UF', 'AGRAVO', 'LOG_UNIVARIADO', 'AIC_UNIVARIADO', 'BIC_UNIVARIADO', 
    'HQIC_UNIVARIADO', 'LJUNGBOX_UNIVARIADO', 'JAQUE_BERA_UNIVARIADO', 
    'KURTOSIS_UNIVARIADO', 'DESCRICAO'
]

for coluna in colunas_necessarias:
    if coluna not in df_resultados_univariado.columns:
        df_resultados_univariado[coluna] = pd.NA

# Salvar resultados em CSV
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_univariados2.csv'
df_resultados_univariado.to_csv(output_path, index=False)

# Exibir resumo
print("\nResultados consolidados salvos em:")
print(output_path)


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.

--- Processando arquivos UNIVARIADOS ---

Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/00B_ts_Dengue_PR_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/00B_ts_Pneumonia_RJ_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_univariada/summaries/00B_ts_Dengue_GO_univariado.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo 

extração exógena


In [0]:
%python
%pip install openpyxl

import os
import pandas as pd
import glob

try:
    import openpyxl
except ImportError:
    print("O módulo 'openpyxl' não está instalado. Use 'pip install openpyxl' para instalá-lo.")
    raise

# Caminho da pasta com arquivos exógenos
path_exogena = '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados exógenos
def extract_exogena_data(file_path):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)

        # Extrair valores baseando-se na posição conhecida
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[19, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[19, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[21, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce')
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce')
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        # Identificar UF e Agravo a partir do nome do arquivo
        filename = os.path.basename(file_path)
        parts = filename.replace(".xlsx", "").split('_')
        uf = parts[-2]
        agravo = " ".join(parts[1:-2])

        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            'LOG_EXOGENA': log_likelihood,
            'AIC_EXOGENA': aic,
            'BIC_EXOGENA': bic,
            'HQIC_EXOGENA': hqic,
            'LJUNGBOX_EXOGENA': ljung_box,
            'JAQUE_BERA_EXOGENA': jarque_bera,
            'KURTOSIS_EXOGENA': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFICATIVA': significancia,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os arquivos da pasta exógena
print("\n--- Processando arquivos EXÓGENOS ---\n")
arquivos_exogena = glob.glob(path_exogena)
for file_path in arquivos_exogena:
    extract_exogena_data(file_path)

# Consolidar resultados
df_resultados_exogena = pd.DataFrame(results_dict.values())

# Verificar e preencher colunas ausentes
colunas_necessarias = [
    'UF', 'AGRAVO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 
    'HQIC_EXOGENA', 'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 
    'KURTOSIS_EXOGENA', 'VARIAVEL_EXOGENA_SIGNIFICATIVA', 
    'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE', 
    'DESCRICAO'
]

for coluna in colunas_necessarias:
    if coluna not in df_resultados_exogena.columns:
        df_resultados_exogena[coluna] = pd.NA

# Salvar resultados em CSV
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_exogena2.csv'
df_resultados_exogena.to_csv(output_path, index=False)

# Exibir resumo
print("\nResultados consolidados salvos em:")
print(output_path)


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.

--- Processando arquivos EXÓGENOS ---

Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Covid-19_CE_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Covid-19_TO_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Cetoacidose diabética_PR_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde 

In [0]:
%python
%pip install openpyxl

import os
import pandas as pd
import glob

try:
    import openpyxl
except ImportError:
    print("O módulo 'openpyxl' não está instalado. Use 'pip install openpyxl' para instalá-lo.")
    raise

# Caminho da pasta com arquivos exógenos
path_exogena = '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados exógenos
def extract_exogena_data(file_path):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)

        # Extrair valores baseando-se na posição conhecida
        log_likelihood = pd.to_numeric(df.iloc[3, 9], errors='coerce')
        aic = pd.to_numeric(df.iloc[4, 6], errors='coerce')
        bic = pd.to_numeric(df.iloc[5, 3], errors='coerce')
        hqic = pd.to_numeric(df.iloc[6, 3], errors='coerce')
        ljung_box = pd.to_numeric(df.iloc[19, 3], errors='coerce')
        jarque_bera = pd.to_numeric(df.iloc[19, 6], errors='coerce')
        kurtosis = pd.to_numeric(df.iloc[22, 4], errors='coerce')
        estatistica = pd.to_numeric(df.iloc[12, 3], errors='coerce')
        pvalue = pd.to_numeric(df.iloc[12, 4], errors='coerce')
        significancia = 1 if pvalue is not None and pvalue <= 0.05 else 0

        # Identificar UF e Agravo a partir do nome do arquivo
        filename = os.path.basename(file_path)
        parts = filename.replace(".xlsx", "").split('_')
        uf = parts[-2]  # Pegando a penúltima parte do nome
        agravo = " ".join(parts[1:-2])

        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            'LOG_EXOGENA': log_likelihood,
            'AIC_EXOGENA': aic,
            'BIC_EXOGENA': bic,
            'HQIC_EXOGENA': hqic,
            'LJUNGBOX_EXOGENA': ljung_box,
            'JAQUE_BERA_EXOGENA': jarque_bera,
            'KURTOSIS_EXOGENA': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFICATIVA': significancia,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os arquivos da pasta exógena
print("\n--- Processando arquivos EXÓGENOS ---\n")
arquivos_exogena = glob.glob(path_exogena)
for file_path in arquivos_exogena:
    extract_exogena_data(file_path)

# Consolidar resultados
df_resultados_exogena = pd.DataFrame(results_dict.values())

# Verificar e preencher colunas ausentes
colunas_necessarias = [
    'UF', 'AGRAVO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 
    'HQIC_EXOGENA', 'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 
    'KURTOSIS_EXOGENA', 'VARIAVEL_EXOGENA_SIGNIFICATIVA', 
    'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE', 
    'DESCRICAO'
]

for coluna in colunas_necessarias:
    if coluna not in df_resultados_exogena.columns:
        df_resultados_exogena[coluna] = pd.NA

# Salvar resultados em CSV
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_exogena_2.csv'
df_resultados_exogena.to_csv(output_path, index=False)

# Exibir resumo
print("\nResultados consolidados salvos em:")
print(output_path)


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.

--- Processando arquivos EXÓGENOS ---

Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Covid-19_CE_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Covid-19_TO_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/ts_Cetoacidose diabética_PR_exaogena_summary.xlsx
Processando arquivo: /Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde 

In [0]:
import pandas as pd

# Caminhos dos arquivos CSV
univariada_path = "/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_univariados2.csv"
exogena_path = "/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_exogena_2.csv"

# Carregar os arquivos CSV
df_univariada = pd.read_csv(univariada_path)
df_exogena = pd.read_csv(exogena_path)

# Separar o UF do AGRAVO na tabela exógena
df_exogena[['AGRAVO', 'UF']] = df_exogena['AGRAVO'].str.extract(r'(.+)\s(\w+)$')

# Definir as colunas finais esperadas
colunas_finais = [
    'UF', 'AGRAVO',
    'LOG_UNIVARIADO', 'AIC_UNIVARIADO', 'BIC_UNIVARIADO', 'HQIC_UNIVARIADO',
    'LJUNGBOX_UNIVARIADO', 'JAQUE_BERA_UNIVARIADO', 'KURTOSIS_UNIVARIADO',
    'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 'HQIC_EXOGENA',
    'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 'KURTOSIS_EXOGENA',
    'VARIAVEL_EXOGENA_SIGNIFICATIVA', 'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE',
    'DESCRICAO'
]

# Fazer o merge das tabelas univariada e exógena
df_final = pd.merge(df_univariada, df_exogena, on=['UF', 'AGRAVO'], how='outer')

# Reindexar para garantir a ordem correta das colunas
df_final = df_final.reindex(columns=colunas_finais)

# Substituir valores NaN por valores padrão (opcional)
df_final.fillna("", inplace=True)

# Salvar a tabela consolidada
output_path = '/Workspace/Users/dayanna.quintanilha@afya.com.br/extracao_final2.csv'
df_final.to_csv(output_path, index=False)

print(f"Tabela consolidada salva em: {output_path}")


Tabela consolidada salva em: /Workspace/Users/dayanna.quintanilha@afya.com.br/extracao_final2.csv


In [0]:
%pip install openpyxl

import os
import pandas as pd
import glob

try:
    import openpyxl
except ImportError:
    print("O módulo 'openpyxl' não está instalado. Use 'pip install openpyxl' para instalá-lo.")
    raise

# Caminho da pasta com arquivos exógenos
path_exogena = '/Workspace/Users/danielly.xavier@iclinic.com.br/Modelo Séries Temporais Clima e Saúde WB/Modelo Séries Temporais Clima e Saúde WB - Resultados/analise_estadual/ts_exogena/summaries/*.xlsx'

# Dicionário para armazenar resultados temporários
results_dict = {}

# Função para extrair dados exógenos
def extract_exogena_data(file_path):
    try:
        print(f"Processando arquivo: {file_path}")
        df = pd.read_excel(file_path, header=None)

        # Extrair valores baseando-se na posição conhecida e garantindo que os números sejam numéricos
        log_likelihood = float(df.iloc[3, 9])
        aic = float(df.iloc[4, 6])
        bic = float(df.iloc[5, 3])
        hqic = float(df.iloc[6, 3])
        ljung_box = float(df.iloc[19, 3])
        jarque_bera = float(df.iloc[19, 6])
        kurtosis = float(df.iloc[21, 4])
        estatistica = float(df.iloc[12, 3])
        pvalue = float(df.iloc[12, 4])
        significancia = 1 if pvalue <= 0.05 else 0

        # Identificar UF e Agravo a partir do nome do arquivo
        filename = os.path.basename(file_path)
        parts = filename.replace(".xlsx", "").split('_')
        uf = parts[-2]
        agravo = " ".join(parts[1:-2])

        descricao = f"Resultados para {agravo} no estado {uf}"

        # Armazenar os dados
        data = {
            'UF': uf,
            'AGRAVO': agravo,
            'LOG_EXOGENA': log_likelihood,
            'AIC_EXOGENA': aic,
            'BIC_EXOGENA': bic,
            'HQIC_EXOGENA': hqic,
            'LJUNGBOX_EXOGENA': ljung_box,
            'JAQUE_BERA_EXOGENA': jarque_bera,
            'KURTOSIS_EXOGENA': kurtosis,
            'VARIAVEL_EXOGENA_SIGNIFICATIVA': significancia,
            'VARIAVEL_EXOGENA_ESTATISTICA': estatistica,
            'VARIAVEL_EXOGENA_PVALUE': pvalue,
            'DESCRICAO': descricao
        }

        # Atualizar o dicionário de resultados
        key = (uf, agravo)
        if key not in results_dict:
            results_dict[key] = {}
        results_dict[key].update(data)

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Processar os arquivos da pasta exógena
print("\n--- Processando arquivos EXÓGENOS ---\n")
arquivos_exogena = glob.glob(path_exogena)
for file_path in arquivos_exogena:
    extract_exogena_data(file_path)

# Consolidar resultados
df_resultados_exogena = pd.DataFrame(results_dict.values())

# Verificar e preencher colunas ausentes
colunas_necessarias = [
    'UF', 'AGRAVO', 'LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 
    'HQIC_EXOGENA', 'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 
    'KURTOSIS_EXOGENA', 'VARIAVEL_EXOGENA_SIGNIFICATIVA', 
    'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE', 
    'DESCRICAO'
]

for coluna in colunas_necessarias:
    if coluna not in df_resultados_exogena.columns:
        df_resultados_exogena[coluna] = pd.NA

# Ajustar a formatação numérica para Excel com separador decimal como vírgula e ponto de milhar
def format_number(value):
    if isinstance(value, (float, int)):
        return f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return value

# Aplicar a formatação nas colunas numéricas
numeric_columns = ['LOG_EXOGENA', 'AIC_EXOGENA', 'BIC_EXOGENA', 'HQIC_EXOGENA',
                   'LJUNGBOX_EXOGENA', 'JAQUE_BERA_EXOGENA', 'KURTOSIS_EXOGENA',
                   'VARIAVEL_EXOGENA_ESTATISTICA', 'VARIAVEL_EXOGENA_PVALUE']
df_resultados_exogena[numeric_columns] = df_resultados_exogena[numeric_columns].applymap(format_number)

# Salvar resultados em CSV
output_path_csv = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_exogena2.csv'
df_resultados_exogena.to_csv(output_path_csv, index=False, sep=";", decimal=",")

# Salvar resultados em Excel
output_path_excel = '/Workspace/Users/dayanna.quintanilha@afya.com.br/resultados_exogena2.xlsx'
df_resultados_exogena.to_excel(output_path_excel, index=False, engine='openpyxl')

# Exibir resumo
print("\nResultados consolidados salvos em:")
print(f"CSV: {output_path_csv}")
print(f"Excel: {output_path_excel}")
